# Text Classification

So most of the machine learning talked about in class has been centered on the notion of allowing a computer to learn from observations. These observations / examples / documents are tagged or labeled data - hence the notion of us performing supervised machine learning since we're supervising the learning by saying that an obersations $x$ from a group of observations $X$ is a actually a label $y$ or better written as

$$y: d \implies c$$

where $y$ is our function for understanding how a document $d$ can be predicted as a specific class $c$.

In the previous lecture session, we talked about how to Naive bayes worked and how we can transform a string (document) into a feature vector. This session will tackle the following:

0.   Initialization (*libraries needed*)
1.   Loading/creating a corpus
2.   Understanding the corpus
3.   Cleaning our raw text data (Pre-processing)
4.   Splitting data into training and testing sets
5.   Extracting features from text
6.   Learning based on the features / Training a model / Testing a model
7.   Meme of the week

And although classification broad problem in NLP, we'll focus on what we were doing before - word sense disambiguation.


# 0. Initialization

So before we get started, let's look at the tools we're going to use.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Brian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. Loading/creating a corpus

So for this session, we're going to go back to one of our earlier examples and try to disambiguate the word sense of "plant" by either meaning the **plant factory** or the **living plant**. To give the proper credit, I took the TWA Sense Tagged dataset accessed at http://lit.eecs.umich.edu/downloads.html#TWA%20Sense%20Tagged%20Data

So let's get started by looking at our corpus



In [2]:
corpus = """factory_sense;"Until the mid- and late 1970s, there were simply no rules whereby groupings of parents could obtain a state-financed, multi-denominational school, as the only channels of communication in these matters were those between the Department of Education and the relevant diocesan department or other church board. For example, a new housing estate would be built, the diocese would be informed of the development at the planning stage, and the diocesan office would put in for a school, or an extension for an existing school. The arrangement had been unchanged since the early years of the state and had become entirely natural. Similarly for the protestant community: the school plant is owned by the church, or appropriate church body."
factory_sense;"Bodfari Foods is a processor and supplier of liquid milk. Last year the company saw pre-tax profits of 1.9m. JUS-ROL INVESTS POTATO product manufacturer Jus-Rol has invested 1.5m in its potato manufacturing plant at Amble, Northumberland. By the end of this month manufacturing capacity is expected to increase three-fold as a result, according to sales and marketing director Joe Weston. IDEAL CHANGE CATERING butcher Ideal Meat Supplies has moved from its Smithfield Market site to premises in North London. The company has also achieved a grade A accreditation from the plant evaluation committee of the National Association of Catering Butchers."
living_sense;"September 1991 1.30 You can win a great new patio Pippa Wood How to cope with a slope Bulbs plant now for spring blooms"
living_sense;"sit like tumors or petrified blossoms between the wire branches and you, an innocent scientist, question me on these brown sparrows: whether we should plant our yards with breadcrumbs"
living_sense;"This plywood structure has grown bigger and bigger, and he has even carved gothic spires on its top. The neighbours complained and when he asked if I minded, I said I loved the kennel and hated the dog. He hasn't talked to me since, but Violet tells me she has explained to him that some people are funny about dogs. She has also warned me that the little porcelain giggling buddha I've put in the tomato plant pot outside my front door will probably be nicked by the Arab boy who delivers the newspapers."
living_sense;"In the brewery the malt is ground into a coarse powder called grist. It is thoroughly mixed in a large vessel called a mash tun with hot pure water. The sugars dissolve into the water and the sweet liquid, called wort, is pumped to a copper. The flowers of the hop plant add bitterness to beer and the oils in the plant also guard against any infections during the brewing process."
factory_sense;"The head brewer at Clark's has helped out a lot, said Ryburn's owner, Harold Coburn. Although small brewers are competitors there's a lot of camaraderie. Harold started Ryburn in July 1990 after being made redundant. He used his pay-off money to buy the five barrel capacity plant."
living_sense;"I've often wondered about the origin of Evan's encyclopaedic knowledge on this topic. It is an area of study with a strong academic bias, the foundations of which were nonetheless in local exploration. This takes back to Thomas Jefferson and William Bingley, who were guided by natives of the region to plants whose efficacy and potency, in the view prevalent at the time, derived from the grandeur of their settlings. (Johnson was an apothecary in search of medicinal plants, and Welsh plant names often reflect their medicinal usage.)"
living_sense;"Protech is a polyester-based film which is easily fitted to flat glass, providing all the benefits of toughened glass at a fraction of the cost. Fitting the film is easy. The glass must be thoroughly cleaned and made dust-free. It is then sprayed with water, for which an ordinary hand-held plant sprayer is ideal."
living_sense;"Please write to this address:  SEEDS OF DESPAIR Dear Dogs Today  I am writing to warn other dog owners of the potential hazard of the rape plant (you'll have noticed the bright yellow flowers now covering so many fields)."
factory_sense;"UCM, a software house from Hove, has developed an animated video showing the workings of the Difference Engine No 2.  RC  Toshiba-IBM LCDs on-stream Display Technologies, a Toshiba-IBM joint venture, has completed construction of its manufacturing plant in Himeji and has started production of large size, LCDs for computer terminals. The plant produces 10.4-in (640 x 480 pixel) LCDs and will gradually add large size units. The plant is said to be a state-of-the-art facility that forms transistors onto large glass substrates, in a class-100 clean room that has the similar level of air cleanliness to those used in the manufacture of LSI chips."
factory_sense;"Every few minutes a truck thunders down the strategic Salang Highway on its way from Hiratan on the Soviet border to Kabul. Convoys half a mile long are watched over by scattered Afghan army outposts on the hills by the road. Drivers report no guerrilla attacks, even though security north of the Salang Tunnel has been largely handed over to local militia units and the army withdrawn. After setting up a bitumen plant and providing earth-moving machinery, the Soviets helped cut a new road east of the old Salang Tunnel."
factory_sense;"But the reason we, unlike the Americans, have no telephones on planes is largely because a consensus on frequencies was a long time coming. Turriff shares get boost from 30% profits rise By FRANK KANE, Chief City Reporter SHARES in Turriff Corporation, the Midlands-based construction and plant hire group, put on 1p to 334p against the market trend on the announcement of a 30 per cent rise in pre-tax profits to 1.76m."
factory_sense;"THE incineration of PCBs requires a furnace which will hold the material at 1,100C for at least two seconds, in the presence of plenty of oxygen. Companies wishing to enter the waste disposal business at this level must demonstrate to the local Waste Disposal Authority (usually a county council, but in some cities and in Wales a more local authority) that they can meet various criteria, although the authority may be over-ruled on appeal to the Secretary of State for the Environment. Her Majesty's Inspectorate of Pollution is now the custodian of national standards on emissions from such plants, of which Britain currently has three. Rechem can take solid and liquid wastes contaminated with PCBs at its plant at Pontypool (30,000 tonnes a year), and liquid wastes (20,000 tonnes a year) at its plant at Fawley, near Southampton. Cleanaway can take liquid PCBs at its Ellesmere Port establishment, where it is building another plant which will take capacity there to 48,000 tonnes annually. Ocean Environmental Management has applied for a 30,000-tonne plant at Seal Sands, Teesside, and is appealing against the local authority's refusal to allow it. Leigh Environmental wants to build an incinerator plant near Doncaster, but also faces local authority opposition, and International Technology Corporation has submitted a joint application with Northumbrian Water for incinerators at Teesside and Tyneside for the incineration of sewage sludge and organic wastes, with a capacity of 43,000 tonnes each."
factory_sense;"The National Cancer Institute was sponsoring more than 30 studies to assess the role of certain nutrients, he said. Radioactive dust forced move By HEATHER MILLS, Courts Correspondent LEVELS OF radioactive dust found in a home near the Sellafield nuclear reprocessing plant in Cumbria were so high that the owners felt they had to move their two young sons to safety immediately, the High Court was told yesterday. But no one wanted to buy Mountain Ash, their four-bedroom house with its panoramic view over the Ravenglass estuary, because of its contamination with plutonium. After two years, Christine and Christopher Merlin were forced to sell the property at auction for 35,500  compared with its valuation of 60,000  to a worker at the nuclear plant."
living_sense;"MANY OF the 60-odd species of slipper orchid, the best known and most popular orchid in the West, could be extinct within 10 years without tough measures to halt the vast illegal trade in wild varieties. The Dutch government is so concerned that at next week's 103-nation conference in Lausanne of the Convention on Trade in Endangered Species (Cites) it will call for a total ban on trade in the plants. It says the present system, which allows controlled international trade, has failed to prevent heavy illegal smuggling of wild Paphiopedilum (slipper) orchids. High prices for rare specimens  one plant recently sold for 3,500 (2,170)  are an obvious incentive."
factory_sense;"Mr Fitton yesterday made clear that he had always been acting in an independent capacity in the offer for Eagle, and that it had no connection with Braithwaite. Whoever sorts Eagle out will make a lot of money, Mr Fitton said. Beneath the can of worms there are some very good businesses. However, Braithwaite, which specialises in plant hire, had tried to take over Samuelson, the film equipment hire company bought by Eagle in 1987."
factory_sense;"He introduced new boilers in the boilerhouse under the main galleries; he installed a completely new steel structure in the ceiling above the entrance hall  without harming the ceiling paintings by Benjamin West and Angelica Kauffmann; he rebuilt the goods lift to carry loads of up to three tonnes into the galleries; he refurbished the public cloakrooms. More recently, he relocated the Schools Canteen from dingy basement to gleaming mezzanine and at the time of his death had nearly completed his greatest coup. This is an ingenious and unique method, devised with Professor Pat O'Sullivan, of environmental tuning in the great Victorian galleries. Rather than imposing artificial conditions, Serjeant allied Victorian engineering to twentieth-century control systems: he deployed the forces of natural convection, rather than cumbersome, mechanically-powered plant, to ensure a regulated flow of humidified and heated air through the galleries."
factory_sense;"By DAVID KEYS A QC claimed yesterday it would be absurd if the law did not allow people forced to leave homes made uninhabitable by radioactive contamination to be compensated. Stephen Sedley was continuing his opening speech on the second day of a damages action against British Nuclear Fuels at the High Court in London. Christopher Merlin, 48, and his wife, Christine, sold their home in Ravenglass, near Sellafield nuclear plant, Cumbria, at a loss after plutonium was found in house dust."
factory_sense;"Beaver hopes to recoup its 7,000 start-up costs within the next six months and possibly sooner if negotiations on a couple of large deals come to fruition. Hewden Stuart offsets problems in housebuilding By ALEXANDRA JACKSON STRONG demand from other segments of the construction market left Hewden Stuart, Britain's largest plant hire group, virtually untouched by current problems in housebuilding in the first half of the year."
factory_sense;"Search operations had to be suspended temporarily at one stage while the roof was demolished because of the danger of it falling in. A British Gas spokesman said investigations into the cause of the blast were continuing. Police released a list of survivors: Henry Wyllie, 20, Helen Lawrie, 48, and Paula Gaunt, 28, were discharged from hospital after treatment; Dawn Howbridge, 19, and Martin Baptie, 19, were said to be satisfactory in hospital. About 1,000 BP employees in Glasgow evacuated their offices for two hours yesterday after a pipe in the chilling plant of an air conditioning system broke loose, releasing gas into the building."
living_sense;"I didn't know The Rolling Stones had started playing just down by the river. Regular correspondence with the revered minimalist La Monte Young introduced him to the Dadaist movement Fluxus, but by the time he had won a Leonard Bernstein scholarship to the Berkshire School of Music in Massachusetts, I felt I'd run my course as far as the avant-garde was concerned. The dream ticket to America led to Lou Reed's primitive melodies and documentary realism, not John Cage. Not that that prevented Cale from screaming at a potted plant sitting on his piano, or suddenly demolishing a table beside his stool with an axe in mid-performance."
factory_sense;"The share advance came against more poor trading news, with Jaguar's US sales down 19.1 per cent last month. Polly Peck, the international trading group, edged ahead 4p to 330p. Shares of its Hong Kong subsidiary were suspended, and after the market closed it was announced that Polly may buy the 30 per cent of the company it does not already own. Cadbury Schweppes fell 6p to 396p as 25 analysts trudged around its Bourneville plant."
factory_sense;"Westland jobs go By LIZ HUNT, Medical Correspondent A further 420 jobs, are to go at Westland, the Somerset helicopter maker, in its high-technology division. The latest redundancies, 345 at Yeovil and 75 at its Bournemouth plant, bring to nearly 3,000 the jobs lost since its rescue in 1986."
living_sense;"IT IS Asian Week at Sotheby's and Christie's. They are mixing Islamic art from the Middle East with Indian and South-East Asian. The sales combine items of charm at fairly accessible prices with some rare treasures aimed at big-time collectors. Homely Persian pottery of the twelfth to thirteenth century combines animal and bird images with sinuous plant motifs and marks the beginning of lustre wares; they are mostly priced in the 500 to 1,000 range and rate high on charm."
living_sense;"The secret  which I now divulge  is that they are dead easy to grow. There is no mystique about their cultivation. Put them in the ground, and up they come. So I urge anyone with a garden to visit Sainsbury's, keep back a few potatoes at the end of the season, before Christmas store them carefully and plant them in the spring."
factory_sense;"There are more than 40 nuclear reactors and 22,000km of gas pipeline passing through built-up areas. The Siberian pipeline disaster, Chernobyl and Bhopal illustrate the potential risks. But the continuous scrutiny demanded and enforced by regulation of the high-risk cases and (for nuclear reactors) the designs of these installations are considerable forces on the side of safety. One day there will be a serious industrial accident; there are occasional near misses and worse; but it usually takes a good measure of folly, inattention and misfortune to overwhelm good plant and a safe system. Relatively few members of the public have been killed in Britain by industrial accidents in recent years. People expect risks they did not ask for and cannot control to be much lower -by 10 or 100 times  than those which they run willingly. People may smoke, drive at 80 miles an hour through fog on a motorway, go in for hang-gliding or rock climbing at the weekends. Yet they see nothing inconsistent with objecting to a well-engineered nuclear waste disposal plant which is clearly necessary to dispose safely of irradiated material already created; or to properly regulated high-combustion chemical waste disposal plants that are the only alternative (unless we get rid of the chemical industry) to toxic wastes being dumped surreptitiously in some field or on some beach."
living_sense;"Today, I've put in a runner bean  it must be all of twenty inches long  but I don't grow them specially. If I see one that looks like it's growing good, I might give it some extra feed and talk to it a bit. But otherwise, he swears by the old tradition of a load of farmyard muck. He sounded as though he disapproved of exotic plant feeds and the new variety of vegetative showmanship."
factory_sense;"In 1985 GM expanded into aerospace technology with the 5bn acquisition of Hughes Aircraft  a diversification which has been copied by Daimler-Benz in its takoever of MBB and, in reverse, by BAe with its purchase of the Rover car group. GM, in common with other Western car manufacturers, perversely views the Japanese automotive industry as both the great collaborator and the great threat in equal measure. This schizophrenic attitude is amply illustrated. On the one hand, GM has joined forces with Toyota in car production and with the Japanese van manufacturer Isuzu, to which it sold a 40 per cent stake in its Bedford van plant in the UK and with which it is now investing 70m to produce a new four-wheel drive vehicle for the European market."
factory_sense;"Spanish, Portuguese, Arabic and Japanese are the most useful languages, Sir Alfred says, because they have a wide catchment area containing many decision-makers who may not have a command of English. Radioactive risks from Sellafield not fully shown By TOM WILKIE, Science Editor RADIOACTIVE contamination from the Sellafield reprocessing plant extends much further than official surveys suggest, according to a report published by Friends of the Earth today."
living_sense;"Ideally, farmers would like to grow the larvae from eggs, and so a new industry  prawn hatcheries  has grown up to meet the demand. There are more than 1,500 hatcheries in Taiwan alone. Hatchery owners then face the problem of getting supplies of the right feedstuffs for their metamorphosing crop. During the larval cycle, prawns change their diet dramatically, from plant plankton to animal plankton and finally to the creatures which live in the sediment."
factory_sense;"However, Lord Marshall's letter says that because of the Government's commitment to introduce competition into electricity supply as quickly as possible alternative arrangements for protecting the security of supply would need to be made, involving a capacity deficiency scheme. Tony Blair, Labour's energy spokesman, last night described Lord Marshall's letter as deeply disturbing and called on John Wakeham, Secretary of State for Energy, for an explanation. A Department of Energy spokesman said: Hopefully this should not lead to power cuts in most cases because distributors ought to be able to get supplies from other area boards. GM may build plant for Jaguar"
factory_sense;"On 22 August, 2,000 fish were killed when chlorinated water from a council swimming pool leaked into the river. More than 120,000 has been paid in compensation to victims of another poisoned water accident in the West Country. South West Water said 477 claims worth 123,644 had been settled, with 318 more still outstanding. Supplies to 20,000 people were poisoned in Cornwall when a relief delivery driver accidentally pumped 20 tonnes of aluminium sulphate into the wrong tank at the Lowermoor treatment plant near Camelford on 6 July 1988."
factory_sense;"But her wider life was anything but solitary. Immensely learned about all aspects of modern Irish life, she knew almost everybody and was known to them. Perhaps the number of thinly-disguised character studies in modern fiction, from Molly Keane to Emma Tennant, are a better tribute to her than anything else, not all of them flattering, it must be said. She was an excellent shrewd local politician and a comfort to her at the end of her life was the defeat of the atrocious plans of the Merrell Dow Chemical Company to build a plant near her beloved River Blackwater, the last campaign in which she played an active part."
factory_sense;"Michael Harrison, our Industrial Editor, contrasts its performance with that of its foreign counterparts By MICHAEL HARRISON, Industrial Editor FORD'S ageing and ramshackle Dagenham site in Essex, once the jewel in the company's overseas operations, enters the 1990s with its very survival uncertain. The plant now dubiously boasts the poorest levels of productivity, quality and working conditions of any Ford factory in Europe. Unless it can effect a remarkable recovery, the factory may well disappear back into the bleak and sodden marshes of Essex from which it emerged nearly 60 years ago. The evidence of Dagenham's predicament can be gauged from internal company documents which illustrate graphically the task confronting Ford and its workforce. These show that it takes Dagenham nearly twice as long to produce a Sierra car as it does Ford's Genk plant in Belgium. The picture is similar when Fiesta production is compared at Dagenham and Valencia. The British plant takes 53 hours to turn out each car, the Spanish factory 30 hours. Product quality at the Essex factory is also inferior, measured against any index, whether it be ratings out of 10, defects per vehicle or warranty costs on each car leaving the factory gate. The Dagenham workforce would argue that there are good reasons why the ageing plant cannot match the performance of its modern Continental rivals. However, the documents also show that Dagenham's car body and assembly operations are failing to meet even its own internal targets. In 1988 the plant was scheduled to produce 992 cars a day."
factory_sense;"Dr Alan Watkins, Hawker Siddeley's chief executive, said the deal with GEC Alsthom would re-inforce its position in the fiercely competitive UK market and enable it to achieve greater exports than the two companies had separately. The company's existing electric motor business, Brook Crompton, and GEC Alsthom Electromotors will have combined sales of about 150m. The GEC Alsthom subsidiary employs 1,400 at three plants in Birmingham, Bradford and Newcastle-under-Lyme. The merged company will supply motors for everything from heaters and hair dryers to large pieces of industrial plant such as powered conveyor belts."
factory_sense;"Rather than censure the dictatorship of President Saddam Hussein, Mr Marlow believes that its poor image is partly the fault of the British press. The press (and television) increasingly make their money (or secure their viewers) by providing entertainment. As for British citizens falling foul of the regime, Mr Marlow believes that No country can be expected to operate a differential system of justice in favour of foreigners, although he adds that benefits can accrue from the exercise of justified clemency. Daphne Parish, a British nurse, has been held incommunicado since last month for giving information on casualties caused by an explosion at an Iraqi missile plant in August."
living_sense;"The exhibition is being organised by Common Ground, a charity, which was created in 1983 to promote new and imaginative approaches to prevent the destruction of the character and fabric of our countryside and towns, writes Joanna Gibbon. The charity sets up projects, often connected with the arts, to persuade people to value their surroundings, regardless of whether they are unusual or not. So far they have set up Parish Maps, New Milestones and recently Trees, Woods and the Green Man projects  the last being connected to the Orchards exhibition. Common Ground is concerned about the decline in old orchards, which grow traditional varieties of fruit, provide habitats for bees, grazing for sheep, geese and pigs and create beautiful landscapes; their exhibition encourages people to grow their own fruit trees, and plant larger orchards for the community."
living_sense;"This move will help us combat the traders putting many rare plants at risk. In another action to conserve endangered plants, the Netherlands  the world's centre for the plants trade  is to second one of its leading botanists to the CITES secretariat to improve the campaign to enforce tighter controls. Dr Ger Van Vliet, scientific director of the Botanical Gardens in Lieden, said his tasks in the next three years would be to make customs officials more aware of endangered plants, reducing the illegal trade, and help countries supplying wild plants to sell correctly propagated specimens. However, Mr van Vliet is at the centre of a disagreement within CITES over how to best protect snowdrops  the genus galathus  another plant worth millions in the trade, which is being endangered by sales of wild stocks."
factory_sense;"The growth in lending in the year to August by Committee of Scottish and London Clearing Banks was only 27 per cent, and for Barclays it was only 10 per cent, she said. In 1980 British banks accounted for 80 per cent of company debt. The figure has fallen to less than 60 per cent as overseas banks have become keen lenders. Chrysler in European car plant talks with Mitsubishi"
living_sense;"Gales in September can tear off leaves before the display begins. The best colour seems to happen after a warm, still Indian summer, terminated by a short, sharp frost. These conditions do not come to order. All we can do is plant and hope."
factory_sense;"A new distribution channel was devised  construction equipment dealers. Despite their lack of familiarity with the product type, the distributors managed to attract customers on the basis of Komatsu's product and price advantages. Levitt considers many Japanese companies as the first truly global marketers and contrasts their practices strongly with the approach taken by Hoover in marketing automatic washing machines in Europe. In the 1960s, Hoover was looking to increase the output of its manufacturing plant closer to its capacity level and so intended to boost its sales in mainland Europe."
living_sense;"Diagonally opposite the stove is the icon corner, although for a Belorussian peasant the most sacred spot is the stove column, where the souls of predecessors reside. Together with the first harvest sheaf hanging beside the icon  for its protective magic, these are reminders that our peasant lives in several cultural times simultaneously, one of them embedded deep in the pagan past, another in the religious present, a third focused on the trip she is about to take to the town of Roslavl'. And away and above her imagination lie the remote worlds of elegant Petrograd, and of Moscow, the city of the Communist future as well as of her inherited religious past. After making first animal feed ( korm ) and only later cabbage soup ( shchi ) for the family, she may finally turn to the work that occupies the womenfolk from November until Lent  spinning flax from that blue-flowered, frail-leaved plant plucked by the root in handfuls in the August of 1921."
living_sense;"Ling At this time of year the moors turn purple with the flowering of heathers. The most common is ling calluna vulgaris . In the past this plant would only have been found on the better drained slopes leading to our moorland plateaux, but because of man's interference in this environment, by digging drainage ditches and lowering the water table, the plant has been able to spread onto the plateaux themselves."
factory_sense;"Certainly, such peace as prevailed owed an immense amount to Jones's personal stature in the movement, as one of the old-style union leaders who had, nevertheless, taken an outward-looking and creative role in trying to relate union demands to a wider pattern of economic stability and also, incidentally, the wellbeing of a Labour government. Jack Jones was an interesting, transitional figure. He was an old leftist who had fought in Spain in the International Brigade. He had grown up in a quasi-syndicalist tradition in the Liverpool docks, and his influence in the sixties had been thrown behind the growth of the shop-stewards movement and local plant bargaining on a devolved basis very much on the lines of the 1968 Donovan Report."
living_sense;"Perhaps there was something more than coherence at stake. A theological college is a narrow world, frequently compared to a greenhouse. After only eighteen months breathing the dust of the docks, Ramsey taught for six years and a half in a greenhouse. If he went to be the head gardener of another greenhouse and did another ten years  teaching among the geraniums, what species of hothouse plant might result?"
factory_sense;"So, What would my priorities be if the bad dream came true, and I were Minister of Health now? If by priorities is meant ambitions for changing the pattern and content of the service, I should be very reluctant to form any priorities. That pattern changes gradually as medical practice and fashions change and as public demand changes. The plant  hospitals, equipment, surgeries  being state-owned and state-administered, those changes do not come about by a gradual process made up of an infinite number of individual decisions: they happen in lurches, of which the most visible form is not the provision of new plant but the discontinuance of old plant."
factory_sense;"There are lots of companies who will take charge of this whole operation, including the printing and fixing of the labels. Alternatively, you can sometimes save money by using different firms for each process. Look in the trade press to find them. If you want a printed sleeve for your record, you need them before the discs are being made so that they can be delivered to the pressing plant and bagged up."
living_sense;"However complex  the outward manifestations of the religious life may be, at bottom it is one and simple. It responds everywhere to one and the same need, and is everywhere derived from one and the same mental state. In all its forms, its object is to raise man above himself and to make him lead a life superior to that which he would lead, if he followed only his own individual whims: beliefs express this life in representations; rites organize it and regulate its working. This passage sheds much light on the method of Mr. Eliot's Sunday Morning Service where the self-mutilation of enervate Origen is placed in the same lineage as the sexual origin of the Word, and where the initial sapient sutlers of the Lord who Drift across the window-panes is a passage with an ambiguous, or better ambivalent, reference, since it holds together in one term both the sable presbyters who bring offerings to church and the bees who bring pollen from one part of the plant to another and so perform the Blest office of the epicene."
factory_sense;"Although it was T U C policy for member unions to refuse government money for ballots, a number of unions defied it. In 1985 the second largest union, the AUEW, was faced with the threat of expulsion from the TUC for breaking the policy. The Electricians Union was  prepared to follow the AUEW and, already faced with the breakaway Democratic Miners' Union in Nottingham, the TUC was forced to find a face-saving formula to avert a possible split. The electricians also incurred the displeasure of fellow trade union leaders by making no-strike agreements with employers, arranging private health care facilities for workers, and then secretly negotiating with the Murdoch press for jobs at the new Wapping plant."
factory_sense;"After he pulled out of the Ryman operation, however, he left manufacturing for good. I was quite sorry to do so because I liked the manufacturing process very much indeed, but I don't believe you can be both a manufacturer and a retailer. You can only be one or the other because the whole business of supply and demand is so complex. When I became chairman of Hepworths, it had its own clothing plant and it either produced too much or not enough of the right quality at the right price at the right time. There's always an argument between your own manufacturing company and the retailing side. Another problem is that if you're trying to deal with other manufacturers in the way that we do, where we have this extremely close relationship and they are very reliant on our forward forecasts of volume, they feel if you have  your own manufacturing plant that you would always give preference to it in bad times and the other suppliers would be the people to suffer if sales declined."
factory_sense;"Such a management style was by no means typical of Nigeria alone. Direct theft from the parastatal system was common in Tanzania by the 1980s. Thus in 1988 the Tanzanian Parliamentary Public Accounts Committee reported that about 5 million had been taken in 1987 in the form of outright theft from parastatal companies. If there is a prevailing thread in the history of these state-owned enterprises it is the constant under-assessment of risk  of the lack of imported materials to supply a manufacturing process, of failure in the supply of inputs ranging from petrol to fertilizer, of farming communities not producing the cash crops to keep a processing plant in operation, and of failing to find competent management for complex operations."
factory_sense;"She had already more than she wanted to deal with. In the silence the sound of the sledge could be heard thudding on stones from one of the near fields. He had already abandoned the timber. Often when talking with the girls she had noticed that whenever Moran entered the room silence and deadness would fall on them; and if he was eating alone or working in the room setting the teeth of a saw, putting a handle in a broken spade on a wet day, taking apart the lighting plant that never seemed to run properly for long  they always tried to slip away."
living_sense;"Helgi dumped the flags and revealed all  he had some farming to do and was going back! Was this guy real? A guide that didn't. He insisted it was simple  go in that direction and plant the flags at regular intervals."
factory_sense;"These injuries went in general unreported in the media. for example, BBC Television on the evening of the 11 July Day of Action, and the papers next morning, were full of pictures of injured policemen, but the pickets who were injured were hardly mentioned, although among them was a man who had had an epileptic fit, a woman who had collapsed at the rear entrance to the factory and two Yorkshire miners who had to be carried away by their friends. Not only was there overwhelming support on the picket line (from trade unionists and from many feminists who identified with the women's movement rather than any particular trade union) but on 15 June local postal workers at Cricklewood started a boycott of Grunwick mail. Because Grunwick was a mail order photoprocessing plant it was totally dependent on postal services."
factory_sense;"Russell Bray  Fewer visitors than ever attended the recent British International Motor Show in Birmingham. The current poor economic climate has been blamed for total admissions of only 420,000 people, some 300,000 down on the 1988 total and the lowest figure since the show moved to the National Exhibition Centre from London in 1978.  Jaguar and GKN's joint venture car components plant under construction at Telford, Shropshire, has been approved for a European Investment Bank loan of 20 million."
factory_sense;"I met Chapman on his way to the Zandvoort prize-giving and said I'd take the job  basically to turn Lotus into an engine manufacturer  if it was still available. Big Lou came out with a story that I'd been fired  but I resigned. Rudd arrived as a powertrain engineering manager at Lotus, producing Elans, Renault-engined Europas and Plus 25s at a rate of 90 a week but selling only 30 a week, largely because of terrible quality problems. Rudd's job was to complete Lotus's new two-litre engine and set up the plant to make it."
living_sense;"I like him a lot. He went round with some seeds, planting them in the ground and that, but some of them grew and some of them didn't. It's always that way  I dunno why, but it is. You can try real hard, but it don't make no difference  sometimes a plant won't grow no matter what you do."
living_sense;"A rhythmic world We live in a rhythmic world. from early childhood we are all aware of the annual cycle of the season and its effect on nature, from the general sense of dormancy in the winter to the burst of light, warmth, colour, and activity in summer. Every year we observe the changes in heat and length of daylight, the migrations of birds, the falling of leaves in Autumn, and the rebirth of plant and animal life in Spring."
factory_sense;"The PTI appeared carrying two footballs and everyone relaxed slightly. He was a good PTI, he made PT fun and didn't just stick to PT and running  but there was no messing about either and he doubled them across the barracks to the football pitch, Where in the next half hour they worked as hard playing football as they would have done in the gymnasium. Meanwhile, as the Quay Foreman (the Warrant Officer in charge of Port Operations), WO2 Barker, walked down the Port South road, five vehicles loaded with ammunition rumbled past him. The duty plant operator, Lance-Corporal Adcock, was just finishing the first parade service on his equipment  a Henley Hermes fork-lift truck with a twenty-five-ton lifting capacity."
living_sense;"Before describing the opportunities and shortcomings of such structures for conversion to domestic use, it is necessary to say something about the process of grinding grain to produce flour, because the anatomy of these buildings was dictated by this process. Corn-milling is one of the oldest and most necessary human occupations. The main good grains on which we depend for an essential part of our daily diet come from plants belonging to the family of grasses and cannot be readily digested until the tough outer shells are broken up and, sometimes, removed. A grain of wheat, for example, consists essentially of three parts: the bran, made up of several outer coverings, the germ (the embryo of the new plant) and the endosperm, the starchy centre which is made into flour."
factory_sense;"Springboard's other activities have included industry representation at careers fairs and conventions, visits to teachers and schools and invitations for schools to visit its offices.  TECHNOLOGY Power to the people Efficient plant is only part of the answer to rising energy costs."
living_sense;"From the Overseas Director's Office Dear YHA member, ETHIOPIA THE MILLION TREE APPEAL Last October, YHA members raised over 47,000 to plant over 800 acres  a staggering 470,000 trees  in Shone Valley, Southern Shewa, Ethiopia. The reforestation programme, targeting to plant 7.5 million trees over a seven year period, is on its way. Soon barren land will begin to show signs of fertility. Then it will be possible for crops to grow on soil where, earlier, n plantlife could have been supported."
factory_sense;"Therefore, how we decide to use our oceans and waterways in future is probably one of the most important choices for this particular generation. The Prince could not have hoped for a better platform. The effect was explosive, and deeply embarrassing to the government's position, which for years had been obstructive  to such an extent that Britain had long been known as The Dirty Man of Europe. Demands for action to clean up sulphur dioxide emissions from power stations, widely believed to be responsible for the acid rain that was killing forests, lakes and rivers  not just in Scotland, but as far afield as Germany and Scandinavia  were brushed aside on the grounds of inconclusive evidence; a similar attitude was taken towards the radioactive discharges from the nuclear reprocessing plant at Sellafield."
living_sense;"WIN A WEEK AT A HEALTH HYDRO Four winners, and their partners, will enjoy a week at top health farm Springs Hydro  plus 100 worth of treatments from beauty specialists DeCleor. You could win a week at Springs Hydro  and bring a friend or partner along with you as a part of the prize. And, as if that weren't enough, while you're there you can enjoy 100 worth of additional treatments from DeCleor, who offer a comprehensive range of aromatherapy and plant-based beauty products."
factory_sense;"The piece prepared by the six young musicians includes a crisp packet being scrunched, playing cards flicked with the skill of a sharper and the zip on 14-year-old Sunjeev Singh's bag buzzing backwards and forwards. But underlying the oddities is a competent musical framework; all six have reached either grade five or six in the conventional music syllabus. They have benefited, too, from Kirklees council's encouragement of music in an area particularly strong in the brass and choral tradition. Fartown's mixture of council estates and tree-lined avenues overlooking ICI's smoky chemical plant provides regular recruits for the Huddersfield choral society and local brass bands."
factory_sense;"NO LONGER can the Government disguise the crisis surrounding electricity privatisation. Ministers have already had to pull Britain's fleet of Magnox reactors out of the float and delay the sell-off date by up to six months. But it has now become clear that further drastic amendments are needed to flotation plans. After months of turning deaf ears to warnings by the City and industry, the Department of Energy has at last conceded that nuclear plant cannot be included in the share float without state support. This leaves the Energy Secretary, Mr John Wakeham, with two options: he can either agree to underwrite any cost escalations in nuclear handling costs after privatisation, or he can swallow his pride, and pull all nuclear plant out of the impending sale."
factory_sense;"A report to be published later this month is understood to detail a series of shortcomings in the rushed sale of the motor company to the group which was approved by Lord Young, the former Trade Secretary. The company, with plants in the West Midlands, Oxford, Glasgow and Llanelli in south Wales, was sold for 150 million which British Aerospace later recouped by floating its half share of DAF-Leyland on the stock market. The report  which examines the civil servants' handling of the sale  is understood to question why British Aerospace was the only main bidder where detailed and full inquiries were made. It also looks at the potential profits which British Aerospace may be able to make by selling redundant sites in Llanelli, Bathgate in Glasgow and the Cowley plant in Oxford which is due to close in the 1990s."
factory_sense;"Coats Viyella says St Austell in Cornwall will lose 290 jobs when it closes its Van Heusen shirt factory there in February. The work will be transferred to the group's factories in Northern Ireland although some workers may find jobs with the group in Plymouth. Last week, Coats Viyella announced nearly 200 job cuts in the North, blaming a combination of pressure from high interest rates, the strong pound and cheap imports. The group is closing the St Austell plant despite recent capital investment and numerous employment initiatives."
factory_sense;"But if, as expected, he fails to gain reselection by his local party next month and carries out his threat to resign and stand against the official Labour candidate in a byelection, he will have to work hard to swing the hearts and minds of many Labour voters. A spot-check of voters walking alongside Birkenhead Park yesterday found Labour voters roughly divided. Mr William Gleeson said: I hope he doesn't lose reselection, but if he does I will still vote for him. He did well for the likes of Cammell Laird and other plant closures, and has done well for the people at Rock Ferry."
living_sense;"PAGE t43 Jekyll by Hyde: Michael Hyde on the life of one of Britain's greatest gardeners, Gertrude Jekyll. Living canvas by Bumps Gardening. ACHINESE proverb has it that if you want to be happy for a week, get married; for a month, kill a pig; for a lifetime, plant a garden."
factory_sense;"Rover believes in catching them young as Judith Jackson discovered in Swindon The body shop Motoring. THE Rover plant at Swindon is not a pretty place. But behind this typical example of dreary industrial architecture, much is going on which should encourage us. For Rover is busy introducing young people to engineering. Engineering is not a glamorous pursuit, nor does Swindon produce the final glossy motor car which might lure potential workers. But the pressings which come out of the plant for Cowley and Longbridge are essential. In an area where unemployment struggles to reach 3 per cent, school leavers are spoiled for choice, so it is in a spirit of enlightened self-interest that the Swindon managers have embarked on programmes designed to tempt young people into engineering. Ray Fletcher, the personnel manager at Rover's Swindon plant and Graham Spaull, the training manager, are concerned that to many youngsters engineering means long hours, tedious jobs and dirty hands. Part of this myth is perpetrated by educationists who have been no nearer a press shop than a picture in a newspaper; parents also see their offspring in cleaner, more respectable jobs. So the success of 20 youngsters on a two-week work experience programme in the Swindon plant on Monday is crucial to the future of engineering in general and the motor industry in particular."
factory_sense;"Well over half of profit still came from overseas, and marine paint and powder coatings were narrowly the largest contributors. In chemicals, cigarette filter acetate kept up its growth and in packaging-film, Cellophane's problems are being countered by a new polypropylene production line. The interim dividend goes up a fraction to 3.1p, from 2.9p. Although interest rates are set to jump after reinvestment in the US and on new plant, earnings of 17.3p, against 17.7p, a share provide a fair base for the two successor companies."
factory_sense;"The island site on the River Seine was a traditional centre of Communist led revolt, notably during the 1936 Popular Front, the 1947 national strikes and the 1968 unrest. The Communist CGT trade union and party leaders warned that they would fight the decision, which threatens 4000 jobs. The party newspaper, L'Humanite, said the firm was giving in to property sharks who want to develop 200 acres around the site where the Renault brothers built their first cars in the family garage more than 90 years ago. The company says the plant, modernised at a cost of 100 million ten years ago, cannot meet today's standards."
factory_sense;"Declaring that the Department of Energy's about-turn will result in merely a temporary interruption to atomic development, chairman Mr Christopher Harding admitted that the nuclear industry's image has been damaged by fears about nuclear waste. Despite the wealth of evidence that nuclear power can never under-price fossil fuel-generated electricity, BNFL is looking at ways of competing for supply in the open market. Proposals involve the possible construction of up to four PWRs on existing BNFL sites at Calder Hall at Sellafield and Chapelcross in Scotland. The new plant would have capacity of between 600 and 2,400 megawatts of power. The company is also planning to expand the life of the existing Magnox reactors by another 10 years. Development costs could be reduced by expanding these existing nuclear sites which are currently used primarily to produce fuel for military purposes. Construction could be caried out as a joint venture with the new state-owned company set up to operate all existing nuclear plant."
living_sense;"What concerns them is the risk that engineered plants might acquire weedy traits and escape from cultivation. The effects on agriculture and the environment could be profound. In a recent report, the Royal Commission on Environmental Pollution concluded that potatoes, for example, might become troublesome weeds if they were engineered to be tolerant of frosts, blight and herbicides. According to ecologist Kathleen Keeler, of the University of Nebraska, the creation of new weeds is the most serious risk attaching to plant biotechnology."
factory_sense;"Companies profit from access to a network of university expertise far beyond individual scheme programmes and graduate associates mature rapidly, Dr Burge. I am frequently amazed at the change in them. Experience from a two-year TC scheme could equal 10 or 15 years on the conventional industrial ladder. Mal Gibb joined a Lancaster scheme after a degree in systems modelling at Sheffield Polytechnic and is now helping to develop computerised production at Nairn Kingfisher, a wallpaper plant on the Lancaster quayside."
factory_sense;"Companies profit from access to a network of university expertise far beyond individual scheme programmes and graduate associates mature rapidly, Dr Burge. I am frequently amazed at the change in them. Experience from a two-year TC scheme could equal 10 or 15 years on the conventional industrial ladder. Mal Gibb joined a Lancaster scheme after a degree in systems modelling at Sheffield Polytechnic and is now helping to develop computerised production at Nairn Kingfisher, a wallpaper plant on the Lancaster quayside."
factory_sense;"Leslie Thomas, aged 27, who organised acid house parties in London, was jailed for five years at the Old Bailey yesterday. Thomas, of east London, plotted to permit premises to be used for the supply of drugs. N-dust judgment delay. Judgment was reserved at the High Court in London yesterday on a claim by Mr Christopher Merlin, aged 48, and his wife, Christine, of Broughton-in-Furness, Cumbria, that their former home was contaminated with plutonimum dust from the Sellafield plant."
factory_sense;"Mrs Thatcher's green conversion has meant she has done more than any other world leader to push the environment to the top of the political, business, and economic agenda, say Friends of the Earth. Page 3. The Guardian. We apologise to those readers who did not receive the Guardian on Saturday when mechanical problems at our London printing plant affected some later editions.."
factory_sense;"Seaq volume at 5pm was 800.4million of which water accounted for 433million. The FTSE Index was 18 points up at 10pm but this was later halved before a later rally saw it close 12.1 points up at 2363.5. The firmer pound helped gilts gain a quarter, but most equities were down. British Aerospace lost 12p later in the day at 561p on market rumours that the Rover 2000 production plant was closing early for Christmas because of low-level business."
factory_sense;"This revealed that the EC would only challenge BAe to honour Rover's plans if there were significant variations, and even then would limit the amount to be reimbursed by the company. Details of the tax deal emerge in a confidential letter from Peat Marwick Mitchell, BAe's tax advisers, showing that the company could offset Rover's losses for tax purposes for three years without fear of a challenge from tax officials. The letter said the Board of Inland Revenue would need to meet before any action could be taken, and BAe and Rover were guaranteed a hearing before action was taken. The Revenue also agreed that if BAe set up a leasing company to take over new Rover plant, rental charges would be tax deductible."
living_sense;"Wales has a wide variety of semi natural vegetation and habitats of considerable nature conservation interest. In the uplands, extensive areas of moorlands, upland grasslands and blanket bog support important communities of floristic, entomological and ornithological value. The higher mountains of Snowdonia are famed for their montane vegetation and relict arctic-alpine plants and insects. Broadleaved woodlands  predominantly oak in the uplands but mixed deciduous in the lowlands, lowland wetlands, lakes and marshes, plant-rich pastures and hay meadows are all important components of the natural environment."
living_sense;"FROYLE Village Hall was ablaze with colour on Friday when the Gardening Club held their annual spring show. The 21 classes were judged by Mr. J. L. Ray, from Basingstoke. Primarily a national vegetable judge, he was more than happy to cast his expert eye over the quality field of spring flowers and he commented on the high standard achieved by the village. He awarded the trophy for the best exhibit in the show to Marian Cresswell for an orange clivia pot plant."
living_sense;"For instance, until we did a NERC-funded project on tortoises in southern France the locals were not involved. Now, ten years later, the tortoise conservation project is self-funding, the toast of the local businessmen, embraced by the local people, forging ahead as one of the most influential conservation outfits in the area, exporting its expertise to other countries, and espoused by the local and national politicians. It is Franco-British co-operation at its best. By manipulating the fungi which exist around the roots of all plants we can make the plant more efficient at taking up  nutrients and make it grow quicker."
living_sense;"But in order to fully answer the question what is aromatherapy?, we need to take a closer look at these amazing aromatic oils. ABOUT ESSENTIAL OILS Essential oils, or essences as they are often called, are the odoriferous, volatile (they readily evaporate) liquid components of aromatic plants. They accumulate in specialised cells or in specific parts of the plant. They may be found in the petals (rose), the leaves (eucalyptus), the wood (sandalwood), the fruit (lemon), the seeds (caraway), roots (sassafras), rhizomes (ginger), resin (pine), gums (frankincense), and sometimes in more than one part of the plant. Lavender, for instance, yields an essential oil from the flowers and leaves. The orange tree is particularly interesting for it produces three different-smelling essences with differing therapeutic properties: neroli (blossom), petit-grain (leaves) and orange (rind). A plant produces essential oils for its own survival: to influence growth and production; to attract pollinating insects; to repel predators; and to protect itself from disease."
factory_sense;"Mary Tavy, Devon MAP A This was an important mining town on the western fringes of Dartmoor in the eighteenth and nineteenth centuries. The Wheal Friendship Mine produced lead, iron, copper and arsenic from right under the main street, and the production of arsenic here went on until 1925. Some remains of the plant are on the eastern outskirts of the village, while a mile to the north, off the A386, the National Trust preserves the former engine house of Wheal Betsy Mine, which produced lead, silver and zinc into the late 1870s, being known then as the Prince Arthur Consols mine."
factory_sense;"That Eden should have allowed the Cabinet a full and lengthy debate on 3 November 1955 could be construed as skilled management, and his referral of the issue for further thought to a Cabinet committee might be seen as prudent policy. His pattern of personal leadership, as revealed by the Cabinet committees he chose to chair himself, reinforces the standard interpretation of him as a man more at ease with overseas than with domestic policy. He followed Churchill's practice and allowed Butler, the Chancellor of the Exchequer, to chair the Economic Policy Committee. Eden, naturally, took Churchill's place at the head of the Defence Committee, and the list of ad hoc committees he chaired reflects both his appetites and the intrinsic importance of their subject matter: GEN 502 on the supply of military aircraft; GEN 503 on security in Cyprus; GEN 506 on preparations for the meeting of foreign  ministers of France, the UK, USSR and USA in Geneva in October 1955; and GEN 511 on a third Indian steel plant."
factory_sense;"Despite the garbled report in The Engineer , Trippier has not tried to impose any tighter deadline than 31 December 1995 for a total ban  which would be almost impossible for UK industry to meet. Austria, Denmark, Germany, the Netherlands and Norway have already unilaterally imposed a 1 January 1995 deadline, and the German chemical industry federation, VCI, has called for a uniform phaseout timetable to be adopted by all signatories to the Montreal Protocol, who will be meeting in Copenhagen later this year. ICI will be able to stop manufacturing CFCs 11 and 12 in the first quarter of next year, having secured supplies for its remaining customers through a deal with Akzo. The Dutch company's small production plant at Weert will be able to continue running efficiently while supplying the diminishing demand for both companies  small amounts of CFCs will still be permitted in essential medical uses after the deadline. ICI has said that it will complete its plant closure programme by the end of 1995. By then, the replacements will be in full production. Last month ICI announced that its third Klea ICI 134a (HFC134a) plant will be built at Mihara, near Hiroshima in Japan. The plant will be built in partnership with Teijin, a long-standing joint venture partner of ICI in agrochemicals. The JV aims to link ICI's process technology and application expertise at its Japan Technical Centre at Tsukuba Science City with Teijin's production experience. The plant will have an initial capacity of 5000 tonnes pa when it comes on stream in 1993, supplying the Asia Pacific refrigeration markets."
factory_sense;"The arrival of 256 miners with problems concerning benefits, mortgage repayments and employment rights necessitated the calling of help from Exeter CAB which worked alongside representatives from building societies and Job Centres. The majority of the miners had never claimed state benefits before and were intimidated by the whole process. When 250 workers were made redundant by Quantock poultry Packers, the CAB moved into the remote Somerset factory. Their bureau is normally 10 miles away in Bridgewater but, with the agreement of Quantock's management, two three-hour weekly sessions were established ten weeks before the closure of the plant."
living_sense;"Even then a shoal can be many hundreds strong. These are waters which do not, as a rule, produce big bream, for with so many mouths to share the available food there is only enough to maintain them at a low body weight. The stronger fish will perhaps reach 4lb. Then there are waters which do not provide the same good spawning facilities; maybe a lack of shallows and plant life in which to lay eggs and milt."
living_sense;"Joseph Gill, John Pinney's hard-drinking cousin, lived as near by as caretaker; Peggy Marsh, described by Dorothy as one of the nicest girls I ever saw, arrived after a month to help in running the enormous house; and from the beginning Wordsworth and Dorothy were preoccupied with caring for a child not yet three years old called Basil Montagu. Basil's widowed father, a natural son of the Earl of Sandwich, formed part of Wordsworth's circle of London  friends, and the proposal that Wordsworth and Dorothy should become responsible for the child was from the beginning an important part of their plans for life at Racedown. He is my perpetual pleasure, Dorothy wrote in March 1796. He is quite metamorphosed from a shivering half starved plant, to a lusty, blooming fearless boy."
factory_sense;"(i) the acquisition of land (including buildings thereon), (ii) the acquisition of vehicles, plant, machinery, etc.,"
living_sense;"plague, pestilence and famine. The prayer that Grandfather Smallweed is remembering occurs in the Litany in The Book of Common Prayer : from plague, pestilence and famine; from battle, and murder, and from sudden death, Good Lord, deliver us. BH  26. plant, the (sl.),"
living_sense;"Living, by necessity, involves delay between desire and fulfilment, between plan and realization. In other words, it involves frustration and the endurance of frustration. There is another popular myth: as long as you love your child all will be well; nature will take its course, the bud will flower, the flower will bloom. If the notion of the sufficiency of love leads to the neglect of social training, it is likely that the young plant (to continue the metaphor) will be blighted."
living_sense;"For more than 50 years I have nurtured what might be called an insatiable interest in China and the arts, crafts, and accomplishments of the Chinese people, the oldest extant civilization on earth. Whether this burning interest came from a transmigration from a previous existence I do not know, but it may have been sparked off by a small and relatively insignificant incident which occurred in the spring of 1929. From time to time, and I think mainly to keep in touch, Aunt Janie would ring me to ask if I could come to the house to help with a small task, such as fixing the cellar door, hammering in a stray loose plank over the cistern and so on  although she must have known as well as I did how useless I was at such household chores. One day she rang me to ask if I could come along on Saturday and help plant potatoes in the back yard; Uncle John is far too busy at this time of the year."
factory_sense;"A total of 22 552 workers were followed up on an average of 18.6 years. Overall mortality was 23% lower than the national average for all causes of death and 18% lower for cancer. For a small group of workers, however, who had experienced above average external exposure and had also been monitored for internal contamination by radionuclides, the mortality from cancer of the prostate gland and the lung was raised (Beral et al, 1988). In workforces in the USA, excess mortality from prostatic cancer has also been reported in the Oak Ridge National Laboratory and in a weapon plant but, Beral et al continue, recent data from the Japanese atomic bomb survivors show no association between a single acute exposure to high levels of radiation and mortality from prostatic cancer. The authors indicate therefore that excesses of prostatic cancer in industrial workforces are unlikely to be due to external radiation. Cancers in other specific body sites were also monitored and compared with surveys from other countries but more research requires to be done. Certainly following the far-flung effects of the disaster at the Chernobyl nuclear power station in 1986, the health hazards associated with nuclear products have been highlighted, not only for employees and people in the immediate environs of a nuclear plant but also for people and terrain far distant from the scene of the nuclear leak."
factory_sense;"For a quick snack I turned to milk chocolate. For me, going on a strict vegan diet where all animal products were banned, where there was no place for tea   or coffee, where even fish, eggs, salt and sugar were forbidden, was like leaping from a warm bed into the icy seas of Antarctica. The idea of the diet was to detoxify the body. For more than sixty years my poor old liver and kidney  1 now had only one  had struggled valiantly to act as a purification plant and sewage works."
living_sense;"as the voices of people who only have talking left Her music plays on, plant, body, animal,"
living_sense;"The latter's definition is adopted here:  the spread of desert-like conditions of low biological productivity to drylands outside the previous desert boundaries. Desertification is the long-term degradation of drylands, resulting either from over-use by man sic and his animals, or from natural causes such as climatic fluctuations. It leads to loss of vegetation cover, loss of topsoil by wind or water erosion, or loss of useful plant production as a result of salinization or excessive sedimentation associated with sand dunes, sand sheets or torrents."
living_sense;"All men go through life clinging to an idea of an earthly paradise. Now we can sail to the Pacific of the mystics, and go beyond the dreams of Cook and Bougainville for we are pilots and navigators of an infinite sea. The whole of the transcendent is virgin land whereas the mundane is filled with the debris of a lifetime. We have been living in a jungle all our lives and we have struggled to bring our minds into cultivation but without hope and suddenly we have found this meadow of cleared land on which we can plant our gardens."
factory_sense;"What we choose to call this event depends upon factors such as its magnitude: we speak for instance of accidents, emergencies, disasters and catastrophes, each of which carries connotations concerning the scale of the event and each of which will have a set of human and/or environmental consequences. On 30 March 1956 one of the most powerful volcanic disturbances this century, the Bezymianny eruption, occurred. Few people noticed the event as it took place in an uninhabited part of Kamchatcka and caused no known casualties. In contrast, the extrusion of a small volume of lava from a secondary cone on the slopes of Tristan da Cunha became the focus of global interest when, during October 1961, the island's lobster processing plant was smothered and the 300 people of the island's only settlement were evacuated to the UK."
living_sense;"Root crops, such as sugarbeet, are hoed  between the rows several times. This helps to control weeds. Peas and beans put nitrogen into the soil. Grass, which is sown with clover , provides rich pasture for cattle in summer and the clover is another plant which puts nitrogen into the soil."
factory_sense;"We've always done it this way is as daft an excuse for an industrial manufacturing process which has become fossilised as it is for saying that fossils have a life of their own. In other words, watching the film persuaded me that despite the intricacy of the life-producing process, it is not wrong to think of helping it along in certain ways. Thus I am not in principle against the idea of research into embryos. However, the second compelling point which struck me in watching the film was a realisation that this was like watching a film of a milk bottling plant."
living_sense;"Katina MacDonald, widow of one of their  descendants  lives there at the present time. Aye, it's a dark street in winter, she says. But the contrast of summer, with the street (the only thoroughfare from pier to shops, Post Office and hotel) filled with visitors, cars, yachties, locals, the odd sheep or two, and the rellies  all packed into the tiny cottages makes both seasons of the year worth waiting for. Katina puts out her best potted plant on a stand on the pavement in the summer."
factory_sense;"Although it is conceded that a decline occurred in a few trades, reformers are said to  have been confused by terminology and to have failed to distinguish between deskilling of individual jobs and industrial changes which led to the number of jobs in certain industries or occupations declining either relatively or absolutely, but not to the nature of the job themselves changing. Other writers have concluded differently, especially with reference to engineering, building, shipbuilding, and printing. Whatever the disagreements, it seems reasonable to say that the period circa  18901914 witnessed an unprecedented advance of machine technique and of mechanisation generally and automatism in particular. There occurred a new era of accuracy and precision in engineering the spread of the interchangeable system increasing subdivision of productive processes and specialization of both plant and labour."
factory_sense;"Labour MPs and some key Conservative figures, such as Michael Heseltine and Leon Brittan, argued that a removal of the automatic grant would worsen regional imbalances (Duffy, 1988). Obviously, companies prefer automatic grant, but that is not a justification from the government's point of view (Begg and McDowell, 1987). This loss of automatic grant might prove particularly important when regions attempt to attract major international inward investments. The Nissan car plant near Sunderland is a notable example of this. In 1988 the production of a second model from the plant was supported by a 25 million grant, which helped to provide another 1,400 jobs."
factory_sense;"Credit standards The existence of money is essential to the modern system of production which depends upon the granting of credit to those who undertake the manufacture of goods in anticipation of sales. It facilitates the development of a sophisticated financial system and thus avoids the need for a pure cash economy, i.e. payment in notes and coin only. It enables capital investment (acquisition of plant and machinery) to take place."
factory_sense;"Now came five tall three storey houses belonging to St. Martin's Church and, although making a terrace, each was slightly different to its neighbour. The properties had been left to the Church in the early eighteenth century and were let at moderate rents. I would imagine that the site had at one time been occupied by a large merchant's house as four of the houses were built over a medieval cellar. This had a separate entrance and had been used for many purposes  the storage of deeds by the Wilts and Dorset Bank, prior to this by Mr. Peru, a solicitor, but when I was a small boy it had been leased to The Haunch of Venison Inn as a wine store and bottling plant."
factory_sense;"The apparently extreme view expressed by Masterman (1987) may perhaps be the public perception of local-authority services as a whole, but equally many people working in schools would also subscribe to this view when discussing the local education authority's administration of the education service. Any superficial investigation into the role of the education officer will dispel the view expressed by Masterman once some appreciation and understanding is gained of the day-to-day duties that are performed both in terms of strategic planning and institutional management. A view from county offices is equally, and often, a mirror image of the incoming vibrations. An ill-informed perception of schools suggests that they need to be protected from the real and harsh decisions involved in administering employment legislation, school meals, plant and buildings: they do not understand what we do, the services we undertake on their behalf, in order that they can get on with teaching the children."
living_sense;"Today there is much within Belfast to attract visitors and citizens alike. There is no more magnificent building than the early 19th century Palm House within the Botanic Gardens. One of the first examples of a curvilinear glasshouse, it stands as a reminder of bygone eras in Belfast's history. The nearby Tropical Ravine is an unique example of Victoriana, and its valuable plant collection provides a captivating and educational experience for many local schoolchildren."
living_sense;"Here at Nottingham it is recognized that the source of our special contribution to academic achievement is the quality of the people who work within it. The interaction of the enquiring and critical minds of students and staff, the pursuit of ideas and the desire to move forward the boundaries of knowledge are the hallmarks of higher education. The University has many staff whose work of outstanding international merit in many diverse fields has brought it great acclaim. Today pioneering work is being carried out in every faculty: engineers work on the motor car engine  botanists on plant breeding  physicists on the production of images of the soft tissue of the human body using revolutionary new techniques  social scientists on the political development of Eastern Europe and on computer systems which mimic human experts  literary scholars produce new insights into the works of major writers."
living_sense;"What is interesting, however, is the importance of clast size in the breakage of bone, and clearly in a cave environment, where limestone clasts are abundant, angular and often very large, small mammal bones are likely to fragment very quickly indeed, in a matter of minutes rather than hours. Modifications after burial After burial, bones are protected from the worst effects of weathering, but they may still be subjected to corrosion by soil or sediment. This may take the form of acid etching in highly acid soils, root marks and corrosion produced by plant roots, corrosion caused by organic acids, and a form of corrosion apparently associated with high humidity in soils and caves."
factory_sense;"If this is accepted then odours per se , destroying as they do any feeling of well-being, the enjoyment of food and home, are prejudicial to health and adequate legal measures ought to be available to ensure their control. Measurement Once again a distinction must be made between an odorant defined earlier as any chemical compound which stimulates the olfactory sense and an odour being a sensation of smell. Instrumental analytical techniques such as gas chromatography combined with mass spectrometry can separate, identify and measure the concentrations of the major components of an odorant, though even this is  fraught  with difficulty as odorous gases from a rendering plant for example have been known to contain more than 30 components."
living_sense;"In the Dinantian rocks which appear to form a major part of the leading Variscan thrust slice, the source potential of the limestones and dolomites (which represent shallow water facies) is assumed on the whole to be relatively low. The underlying Lower Limestone Shale is also interpreted as a shallow water sequence, but some beds are believed to have been laid down in poorly aerated conditions (George 1958) and may therefore contain source material. Further south, deeper water conditions, and hence the possibility of preservation of organic matter, persisted from the Devonian into the Early Carboniferous. During the Late Carboniferous the proliferation of plant growth on land not only provided coal seams but contributed abundant particulate and dissolved organic matter to the pro-delta and deeper marine areas."
factory_sense;"With the absorption of other lines into the system, together with carriage building shops, the number of branch works became too scattered, so they were all amalgamated, and works leased at Saltley, Birmingham. There the work was carried out until the L  B Railway had amalgamated with and leased a number of small lines, notably the Birmingham and Manchester, Liverpool and Manchester, and the South Staffordshire Railways. Wolverton, which had been the seat of the locomotive building industry for the L  B Railway was no longer so convenient as when the northern terminus was at Birmingham. The locomotive works at Crewe now provided engines for the Northern Division and, in 1 8 65, some of the Wolverton Shops were vacated, the Carriage Works at Saltley abandoned, and the whole of the plant removed to Wolverton."
factory_sense;"The students spend a given amount of time per week for a period of months, with the guidance of an assigned individual in the company and with the support of the technical staff of the school or college. At the end of the process students present their solution to a panel of mixed experts and lay people, and answer questions about the engineering issues as well as matters to do with team-building, leadership, decision-making and planning. The main objective is to find a solution agreeable to the company in terms of its feasibility and cost. In 1990 a group of students worked with Rolls Royce at the aeroengine plant in Barnoldswick, Lancashire."
living_sense;"You give the assistant 10 and she gives you 1.30 change. Is she right? 5 A plant costs 3.25, a bunch of flowers 95p and a pack of bulbs 1.89."
living_sense;"Thus grassland (especially wet) is an important habitat for many specialised bird species, and its loss has reduced and restricted their populations. Other habitat losses have resulted from farm mechanisation which, requiring larger and more conveniently-shaped fields, has resulted in much hedge and scrub clearance. The more satisfactory economic position of farming allied with Government policies have also encouraged these losses. There have been dramatic declines of a few bird species as a result of using some chemicals, and perhaps more subtle and long-term changes as a result of the simplification of the plant community of farmland arising from cropping changes and the widespread use of increasingly sophisticated herbicides."
living_sense;"During the discussion on foregrounds, he frequently comments on how the appearance of an area could be improved by thoughtful planting. His pen lashed friend and foe alike. Miss Pritchard of Croft Lodge at Clappersgate, where he was a frequent visitor, was set out in the book as having lands too thick with trees. His host of many a good evening, Mr. Harden, at Brathay Hall, was set up as an example of one who did not plant enough trees."
living_sense;"With a quick reflex action he caught the tompaw, a short stout wooden handle with a steel claw set in the end, and thumped it into the first deal of the day. Yanto worked with a will. Making up the slings of timber ready for Tiny Bennion's crane to pick up and dump on the wharf. The other half of the gang, working on the wharf, then took the individual timbers on their leather padded shoulders, and with that strange loping run, their steps synchronised to the bending motion of the plant, would deposit them into the timber sheds."
living_sense;"Throughout the nursery and garden we emphasise plants suitable for cutting flowers or foliage for indoor decoration. We have shrubs and herbaceous perennials which provide foliage for arrangements all year round as well as flowers in season. There is a selection of perennials, biennials and annuals which produce abundant flowers for cutting, many of which are also ideal for drying. We give tips, as recommended by the R.H.S. at Wisley, on conditioning plant material for use in arrangements and on preserving material for use out of season or in more permanent arrangements."
factory_sense;"White says that to be a clavicle, the specimen should have an S- or sigmoid curie, but it does not. Also, a tiny opening in the bone, called the  neutrient  foramen, should open laterally, but it opens in the opposite direction. Seveso's designer claims modification caused explosion THE FACTORY explosion at Seveso, northern Italy, which  polluted  the town with dioxin in 1976, was caused by negligence, according to the engineer who designed the chemical plant. Fritz Moeri, one of five people charged in the trial of company officials which began on 18 April, has told New Scientist  that the disaster would have been impossible if the system he built in 1970 had not been modified and if fashionable exhaust valves had not been added at the insistence of company management. The accident happened when workers at the ICMESA plant neglected to add water to stop a reaction producing trichlorophenol (TCP) as they went off shift early one Saturday morning. The reactor's contents spontaneously over-heated, expanded, blew valves on vents and emptied the contents of the cauldron into the air above the plant."
factory_sense;"This, the study concluded, indicated that there was no evidence that army service in Vietnam had increased the risk of birth defects in veterans' children. Members of the Vietnam Veterans Association have rejected the findings and called for a Royal Commission into the issue. Choosy over Chooz THE BELGIAN government has put off a decision on whether to stump up cash for the nuclear plant being built by France at Chooz  a finger of France sucking into Belgian territory."
factory_sense;"The company chose Wrexham partly because of the generous financial aid available. The British government's Welsh Office says the project was the culmination of discussions started in 1980 when Edwards led an industrial mission to the US. The Welsh Development Agency says that to ensure that the Wrexham project got under way as quickly as possible it cooperated fully. Wrexham is a special development area, so Intermagnetics is entitled to a regional development grant of 22 per cent on all capital expenditure on plant, machinery and buildings."
factory_sense;"Suitable hard argillaceous rocks  mudstones and shales  are found in Wales, Devon, the Lake District and the southern uplands of Scotland, says the report. For these rocks there is no need for a coastal site. A third idea, to put waste in existing man-made caverns in chalk or limestone, is still being assessed. Such caverns are available in Kent, Wiltshire, Cleveland, Humberside and Lincolnshire, The UKAEA's report proposes that special trains with 1012 wagons, each loaded with flasks containing drums of waste will travel on public railways between the Sellafield reprocessing plant and the waste dumps, which could be up to 400 km away."
factory_sense;"We are totally against any form of discrimination, a Belgian official told New Scientist . Such freedom for Britain and France would stymie the EEC's efforts to ensure equitable shares of fuel if shortages occur in future. Observers also point out that the changes would allow electricity authorities in Britain and France to conceal direct or indirect military subsidies to nuclear power. In practice the two industries are very integrated, sharing plant and facilities."
factory_sense;"New dump for nuclear waste is planned THE CENTRAL Electricity Generating Board wants to build a giant nuclear dump to store spent fuel from its nuclear power stations. The dump will hold the highly-radioactive waste until it can be taken to British Nuclear Fuel's Sellafield works for reprocessing to extract uranium and plutonium. The dump is necessary because BBNFL's new THORP reprocessing plant, which should be in operation by 1990, will not be able to handle the volumes of waste that will be produced by Britain's power stations in the mid-1990s."
factory_sense;"Italy wants to dump Seveso's dioxin Brian Price  POISONOUS  dioxin waste may be dumped in the Atlantic if proposals from the Italian government are accepted by the London Dumping Convention next week. The waste comes from Seveso, near Milan, where an accident at a pesticide plant in 1976 contaminated a wide area with dioxin (TCDD). On 1 October the Italians proposed to the convention's scientific committee that the reactor pot, which was at the heart of the explosion, should be dumped, 700 km off the Spanish coast in the area used annually by Britain for dumping nuclear waste. But rumour is rife that the reactor pot has already been buried. But the rumours are denied by the plant's owner, Hoffman-LaRoche, which says that the reactor is still being dismantled inside the sealed-off plant."
factory_sense;"THIS WEEK Sizewell safety remains shrouded in mystery SIX WEEKS after the Sizewell inquiry opened, key safety issues surrounding the plan for a pressurised-water reactor (PWR) remain unresolved. And the technical reliability of a crucial design feature of the plant  its steam-generators  remain a matter of conjecture."
factory_sense;"One activity that has already been set up is the international Intor workshop, in which groups from the US, Europe, japan and the USSR have met to establish a common set of objectives and an optimum design for demonstrating the technology needed for a fusion reactor based on the tokamak system. The International Atomic Energy Agency of the United Nations organises the Intor workshops (Intor stands for International Tokamak Reactor) although most of the work is actually carried out by groups working in their various home laboratories. The leading groups meet in Vienna periodically to bring the work together. The Intor workshop has already produced a first broad proposal a definition of the envisaged plant and its objectives. Intor is presently conceived as a 600 MW (thermal) reactor that would run for 10 years, researching and, we hope, demonstrating the practicability of repeated long pulses burning thermonuclear fuel in a tokamak. Intor could also test the life of components in the flux of neutrons  at an energy of 14 MeV  produced by reactions between deuterium and tritium. Such a device would become radioactive because of those neutrons and Intor would also have to demonstrate the practicability of maintenance by remote control. Intor would not produce electricity, that would come later in the so-called demonstration plant."
factory_sense;"BY NOW Sir Frank Layfield QC will be well aware of the task that is before him. It is nearly a year and a half since David Howell, the then Secretary of State for Energy, announced that Sir Frank would be the inspector at the public inquiry into the plans of the Central Electricity Generating Board to build a pressurised-water reactor (PWR) at Sizewell in Suffolk, and there has been a veritable deluge of documentation from all sides in the past 18 months, giving Sir Frank more than enough preliminary reading. There are various theories as to how long the inquiry will last, ranging from four months to nine months, or even longer. The shorter estimate comes from one or two optimists within the electricity supply industry who believe that the opponents to the plans to build the Sizewell PWR will prove less persistent than they were at the inquiry into the plans to expand the nuclear fuel plant at Windscale in Cumbria."
living_sense;"As they say elsewhere watch this space. THE MOST NEGLECTED WILDLIFE HABITAT OF ALL Jenny Owen You may live within a few metres of many if not most of the plant and animal species ever recorded in Britain, and perhaps a few that are new to science."
factory_sense;"Convening production lines to largely automated production of flat panel sets would be costly in the short term. It would cause lay-offs in Japan and licensed manufacturing outlets elsewhere. Sony expects that CRT may increase its workforce depending on market interest in televisions. Sinclair with his similar, hut cheaper set also predicts great things despite industrial hiccups at his Dundee production plant."
factory_sense;"The figure of 180 000 tonnes refer to almond kernels, which make up about 60 per cent of the total weight in California. (In the Mediterranean the kernel is only 35 per cent and the shell 65 per cent; the Californian reversal is just one result of the intensive breeding and selection programme.) So there are about 120000 tonnes of shell, and the frugal California Almond Growers Exchange uses it to generate electricity. The shells power all the Exchange's plant, and also provides a small surplus that is fed into the local grid."
living_sense;"But we have to look carefully and dispassionately to find them. We can classify the truly effective tonics into two groups: those of traditional herbal origin, and those made or isolated by chemicals in more recent times. Of the herbal tonics and restoratives, we have lost many. But one that has surfaced in recent years is ginseng, the root of the araliaceous plant, Panax ginseng . This plant, a relative of ivy and indigenous to the temperate mountains of the Far East, has been used continually for some 5000 years in the Orient as a tonic, restorative and preventive remedy."
factory_sense;"When the old industries die they are usually operating near to peak employment, explains James Wilson, the corporation's chief executive. The new firms start small and will not reach a peak for several years. Furthermore, the numbers of jobs created per pound of investment is, often startlingly low. Nippon Electric of Japan has recently opened a 50 million semiconductor factory in Livingston that will, by 1985, when the plant is operating at full stretch, employ only 800 people. A few months ago the plant recruited its first workers  the men and women, mainly aged under 25, whose minds and fingers are agile enough to turn out microscopic silicon chips by the hundreds of thousands."
factory_sense;"It also needs a second additive to help it dissolve and raises the vapour pressure of petrol, which can cause havoc with a car's Fuel pumping system. MTBE is more expensive but better behaved. In fact Shell, Britain's leading oil company, marketed ether blends of petrol in the 1950s for hot-rod enthusiasts. At the moment the entire European production of MTBE is around 500000 tonnes per year, though this will double if a company called Highland Hydrocarbon decides to go ahead with its plan for a new plant making MTBE from North Sea gas at Nigg Bay in Scotland."
factory_sense;"Passions have cooled since then, and the committee now recommends that NASA consider building instruments especially for hitchhiking through the galaxy on ESA spacecraft. Dealing escalates as Seveso trial opens THE TRIAL of technicians who let the chemical works at Seveso in northern Italy run out of control seven years ago, scattering 300 grams of dioxin across the district, opened on Monday in Monza, Italy. But the judge ordered a three-week postponement while the plant's owner, the Swiss flavouring and fragrance firm, Givaudan, tries to reach settlement with the 127 representatives of local citizens and organisations who flooded the court asking for compensation."
factory_sense;"At an ARC symposium last week, convened to mark the ARC's decision to rename itself the Agricultural and Food Research Council, delegates called for action to save the British malting industry which is hit by falling demand for Scotch whisky. Delegates called for the ARC to conduct urgent genetic research to find new uses for the products of the malt and barley industries. Sloppy clean-up THE fourth anniversary of the accident at the Three Mile Island nuclear reactor in the US passes, the crippled nuclear plant is once again the centre of controversy. This time the issue is the safety of the clean-up of the plant. During the past fortnight engineers working on the site have charged, in affidavits, that the efforts to remove damaged uranium fuel from the reactor are sloppy. The present mentality on the island emphasises short cuts, expedience and disdain for professional standard, said Edwin Gishel, engineering director for site operations in an affidavit sent last week to his employer, the General Public Utilities Corporation. And Richard Parks, an engineer with the Bechtel Corporation, which built the plant and is the prime contractor for the clean-up, claimed that the operation is disorganised and at times irresponsible."
factory_sense;"Council officers are especially worried about the reviewers' recommendation that the statutory criteria for selecting SSSIs  which are at present purely scientific  should take into account administration and finance. Reprocessing recruit BELGIUM is to rejoin the nuclear fuel reprocessing race. A project to revive its Eurochemic plant for reprocessing nuclear fuel, at Mol in the northeast of the country, has just obtained the senate's approval. It should be fully operational by 1986. The plant will initially reprocess irradiated fuels from the country's five existing nuclear power plants. The plant was built by the Nuclear Energy Agency of the OECD, which set up Eurochemic in 1957. In 1974 it was squeezed out of the market by an Anglo-French-German enterprise, United Reprocessors. Both France and West Germany intend to double their reprocessing capacity by 1990  when the THORP plant at Windscale should be in business."
factory_sense;"The appeal has been five years in the hearing. But the plaintiffs, local authorities and anti-nuclear associations must have abandoned hope of success long ago. For Electricite de France carried on building throughout the appeals procedure and the first Cruas reactor is due to be coupled to the national grid in two months. The protesters  appealed  against both the authorisation to site a plant at Cruas and against a building permit."
factory_sense;"Storage leaves open the option of reprocessing later, he says, if the value of the uranium and plutonium that could be reclaimed in the process, made it worthwhile. Deacon's plan would involve every nuclear power station in Britain becoming a mini-Windscale, storing many years worth of spent fuel on-site. But the CEGB prefers the idea of a central store. The board says it needs such a store in any case in the 1990s because there may be a shortage of reprocessing capacity at British Nuclear Fuels' Sellafield plant ( New Scientist , 3 February, p 289). But the new admission that Britain is considering (forgetting) about further reprocessing plant opens up the possibility that the temporary store may become a long-term dumping ground for the industry's most dangerous waste."
factory_sense;"Three Westinghouse designed PWRs have been built there since 1974 and a fourth will be running by 1985. The trouble at Doel began last November when workmen refuelling Doel 2 dropped some pieces of metal into the reactor's core. No one noticed for a month, until the contamination showed up at the filters on the primary water circuit. The plant's operators decided to shut down the reactor and clean the rods. After a three-month stoppage, engineers said last week that they expected to have the plant running again in a fortnight."
factory_sense;"(Nathan had passed it once, and remembered a sky lit by ragged flames, torches held aloft by the refineries.) He'd been some kind of engineer. Fifteen years working for the same company. Then a merger, cutbacks at the plant, and he was out of a job."
living_sense;"It was originally set up to help Queen Alexandra's district nurses but now also supports the Macmillan Nurses, cancer charities and the Gardeners' Benevolent Society. Mrs. Foulsham's slides included Vann at Chiddingfold, the planned venue for a summer visit, together with attractive views of her own garden at Albury, Brook Lodge Farm Cottage near Godalming, and High Meadows, Churt. Especially interesting was the garden of Upton Grey where the new owners are bringing Gertrude Jekyll's original design back to its former glory. The society will be staging their spring show at the village hall on April 11th, with a plant show on May 9th."
living_sense;"WATER HOOD FITCH, A CELEBRATION By Jane Lewis  HMSO Books, 17.95 The 19th century explorer and illustrator, Walter Hood Fitch is best known for his flamboyant yet accurate drawings which have made a lasting contribution to our knowledge of the plant world and to the art of botanical illustration."
living_sense;"Down Having a lot of bumps What people do in church Low woody plant or bush"
living_sense;"Next week: drugs for ectopic pregnancy ALTERNATIVE HEALTH Aromatherapy for the elderly Aromatherapy is the use of natural plant essences in healing, relaxation and the reduction of stress."
living_sense;"There were shaky cane chairs to go with the table, which I carried home on my head, and there Mum sat hour after hour, doing calligraphy  Christmas and birthday cards on squares of lush paper. She cleaned as never before, with care and interest (this wasn't a chore now), getting on to her knees with a scrubbing brush and bowl of water, behind cupboards and along skirting boards. She washed down the walls  and repainted doors smudged with our fingerprints. She repotted every plant in the house and started listening to opera."
living_sense;"We shall see that the position has changed slightly, but encouragingly, since this challenge was given. My own burden in writing this book is that as wide a circle of believers as possible consider biblical and practical principles for church planting that will enable us to promote and plan for growth more effectively. But first we need to ask: is church planting an effective means to the growth of the church? We need to answer the question: why plant churches today?  Why plant churches today?"
living_sense;"Viticulture was then unknown in its cold northern reaches: according to the observations of Dioderus Siculus, a Greek historian of the first century BC, the frigid climate of the uncultivated North forbade production of either wine or oil. By the first century AD, however, the Romans were cultivating the vine in Northern Gaul. We know from Cato the Censor (234149 BC), the author of De agri cultura , that the various methods of planting, grafting, training and pruning of the vine, which the Romans would have introduced into Champagne, had been in common use throughout the warmer parts of the empire for at least two hundred years. Without a doubt, the Romans had achieved a sophisticated level of vine-growing, training the vines, much as today, in parallel rows, and ensuring proper spacing between each plant."
factory_sense;"A stocky toddler and his sumo stomp green ideograms of a swiss cheese plant"
living_sense;"It is mentioned in the Domesday Book of 1086; indeed evidence seems to indicate that it was one of the oldest and largest settlements at the time of Domesday. At the time of the Armada in the reign of Elizabeth I, Mappleton had three beacons to light to give warning of invasion. The village is mainly given to agriculture, though tourism will play an increasing part in the future. A farmer's wife of Middle Farm has over the years developed a very attractive plant and shrub centre with success."
living_sense;"However, we hope that you will take time to read the chapters that precede the recipes, menus and diet charts. Because if you do, you cannot fail to be deeply impressed by the very many ways in which high-fibre foods can help you to slim, and to come to the conclusion that the F-Plan is that major slimming breakthrough everyone has been seeking for so long.   What (and where) is dietary fibre? Dietary fibre is a substance obtained from plant foods, as distinct from animal foods. All cereals, fruits and vegetables contain some dietary fibre, but, just as the calorie content of different foods varies to a great degree, so does the fibre content of different plant foods."
living_sense;"Lighting is only necessary for the viewer and the plants, so select  the dimmest that will suffice. Interior decoration also falls into my acceptable inhabitant category. Being a predator, the Tiger Fish requires secluded hiding places, I use Curiowood, and a dense growth of giant Vallis along the background. any tall plant will do  whatever you prefer."
living_sense;"Right: While floating food provides much of these Koi's vitamin requirements. B Bacteria Probably the most diverse group of living organisms on Earth, the Bacteria belong to the plant kingdom, ranking lowest in its orders of species."
living_sense;"So early oxygen levels would have been periodically depressed, in order to allow the first photosynthesizers to carry on evolving. Nevertheless vegetation soon started to proliferate  because liverworts, mosses, ferns, coniferous plants and flowers appeared in quick succession, introducing carbon dioxide into the atmosphere in ever-increasing quantities. This was achieved through photosynthesis: plants were    able, by converting sunlight into energy, to live solely on sunshine, air and water, combined with other simple elements gleaned from the soil. They produce oxygen as a waste product, a normally lethal poison to the plant cell."
living_sense;"Gandhi recognizes this fact. It is impossible to sustain one's body without the destruction of other bodies to some extent. We destroy as much life as we think necessary for our own sustenance and preservation. We destroy plant and may be animal life for food."
living_sense;"I would like to hear from any other readers who share my interest in exotic foliage plants and perhaps we could swap cuttings and propagating material. I am especially interested in growing passion flowers with attractive foliage as they seem to be happy grown in reduced light conditions, therefore making excellent house plants. THE HAPPY KIDS Proculture Plants, leading plant specialists, have introduced a new venture in support of the National Society for the Prevention of Cruelty to Children (NSPCC). Under the logo The Happy Kids, designed by the NSPCC for a range of consumer products, Proculture Plants are offering a range of indoor and outdoor Happy Kids plants. For every plant purchase from Proculture's extensive range, which includes houseplants, cyclamen and perennials, they will be making a donation to the NSPCC."
living_sense;"Rhubarb, rhubarb forcing ensures an earlier yield. Big-hearted: globe artichokes If you doubt that vegetables can be as attractive as flowers, then take a closer look at globe artichokes. This perennial plant's big, bold, green clumps, measuring 90cm/3ft high and the same in diameter, grow even bigger and more productive before they deteriorate after a few years' growth, and are then best replaced."
factory_sense;"Although Stevie and I had known each other for a while we had never been romantically involved. Then somehow we did get romantically linked up and some people, right near the end of the situation with that band, singled us out as potentially more interesting than the band, which I suppose was a cruel situation really. By this time I had already been fooling around with sound on sound recording, and then in 1971 some aunt that I never even knew left a small house to me and my two brothers. The house was sold off and I ended up inheriting 12,000 dollars, so I went straight out and bought an AG 440 Ampex professional four track machine and took it up to my father's coffee plant, and every night when he left I would drive up and work on Stevie's and my songs."
factory_sense;"VIVA EGGLE! Dear Guitarist Following your enthusiastic review of the Patrick Eggle Berlin Pro in March, I was motivated to try one and subsequently purchased a deluxe model with non standard options at very reasonable cost. I travelled to the plant to collect the guitar and was treated to a comprehensive tour of what I can confirm as an impressive facility of skilled and committed people in every department."
living_sense;"It aims to reduce the swelling of the hair during the perming process, meaning the hair's structure is protected from any long term damage. To finish, Ian styled Sue's hair using Natural Styling Shaping Mousse to create this soft natural look and New Style Shaping Hairspray for her glamorous upswept style.   After five years of intensive research, Schwarzkopf have developed Igora Botanic. Designed as an alternative to traditional colouring products, this unique range of natural plant based hair colours is totally chemical free, biodegradable and is leading the way in professional, natural hair colourants."
living_sense;"The alert, attentive mind, the intense concentration needed, is very demanding. When people begin speechreading instruction, their eagerness to progress and to understand everything instantly makes them very vulnerable to fatigue. They may strain forward with muscles tense, hands gripped until the knuckles show white through the skin, and eyes staring unblinkingly. It is not easy to accept that, like a plant that grows, the process may be gradual (although sometimes this is not the case)."
living_sense;"We need a positive-images campaign which is positive about all  our images. SEXUAL ABSENCE 5 Wings beating against glass/a bleeding that never ends/desire is endless corridors leading nowhere. A plant you don't water, it droops but it does not die/your collection of erotic images is considerable but it's always happening to someone else."
living_sense;"As we go up the scale of complexity, many highly evolved and sophisticated molecules are involved in the constitution of even the simplest living cell. The progression from the molecule to the cell is also the leap from the non-living to the living world. With our present-day highly developed technologies, we can re-create many of the complex molecules found in nature, but we cannot create life  another  argument in favour of the existence of the life force or life energy, whatever it may be. Cells organize themselves together to form tissues such as fatty tissue, muscle or bone; tissues organize to form organs, such as the heart, lungs or brain; many organs and tissues, working together, form living bodies, be they plant, animal or human."
living_sense;"The Consumers' Association says that manufacturers aren't helping the situation by labelling clothes dry-clean only to protect themselves when many could be washed with care. A BREATH OF FRESH AIR Houseplants not only make our home or office look more cheery but, according to research by NASA (the National Aeronautics and Space Administration) they are also sophisticated weapons against pollution. NASA has found that between 15 and 20 plants, such as the humble spider plant, can clean and refresh the air in an average-sized room."
factory_sense;"Axle works hit More job losses at Leyland DAF plants RECEIVERS controlling the British arm of the collapsed truck and van company Leyland DAF announced a further 349 job cuts yesterday. The new cuts  273 at the truck plants in Leyland, Lancs, and 76 at the Albion Axle plant in Glasgow  take the total number of redundancies since the company collapsed earlier this year to almost 2,300."
factory_sense;"Court setback to Daf survival bid ATTEMPTS to save the British section of lorry manufacturer Leyland Daf from closure suffered a setback in the High Court yesterday. The company, which has been in the hands of receivers since February, faces the prospect of its production lines halting after a judge refused to rule that a crucial supplier, to whom it owes large sums of money, was legally obliged to continue providing vital components. It is understood that the Birmingham-based van operation and Leyland lorry assembly plant in Lancashire have only enough brake and clutch assemblies to last them a matter of days."
factory_sense;"Royal July visit Princess to keep a double appointment A VISIT by the Princess of Wales to Suffolk's largest engineering works will mean a Royal chapter to complete its 50-year history. The Princess will meet several hundred workers when she tours the Lucas Diesel Systems plant at Sudbury on July 27."
living_sense;"This meant that I was free to respond to the effects that developed as I worked and I got a feel both for the subject and the pastels I was using. the arrangement was placed low down so that the angle of view presented me this as interesting a set of shapes as possible.  FIGURE 5 After the initial laying in of colour I began to introduce more of a feeling of form to the objects while retaining the strong sense of pattern To begin with I made a rough drawing and then began to place the colours using the nearest equivalent values from a set of 36 Polychromos Pastels: blue violet, No. 137, lemon, No. 107 and night green, No. 155, for the mass colouring of the plant; an African violet, rose carmine, No. 124, light blue, No. 147, and blue violet, No. 137 again for the silk scarf; orange yellow, No. 109 and orange light, No. 113 for the vase and so on, until I had an abstract pattern of colour distilled from the still life subject."
factory_sense;"They have been formed by the hydrothermal stoping of limestone causing collapse breccias which were subsequently replaced by fluorite. They show similarities to MVT deposits in the USA as described by Briskey and others (1986). There is potential for further discoveries. There are also several small tributers feeding a central processing plant near Stoney Middleton."
living_sense;"I think you'll find put some solvent into it  it would erm   Meing  Does that mean smelly? So I'm not going to show  don't confuse that with this which is  oil  derived from  petroleum products. Linseed oil is mostly made out of the seeds of the  lin  linen plant."
living_sense;"Er, one of the things it won't tell you on here, whether it's a blazing hot day, whether there's snow on the ground or whether it's just pouring, dull and people  are coming along. So there's, there are a number of variables which we, we will never know about.  But I think Colin your main concern wasn't it that perhaps we weren't  hitting the right slots and ? Use of plant I think was one of the things you were concerned about? Sorry? Use of plant I think really, wasn't it?"
factory_sense;"that er nuclear, nuclear yes it's Chernobyl, he was looking for the real word there, a nuclear what? A nuclear power station Alright, no, but the actual ray of the, of the plant, the nuclear reactor, right what, what happened?"
factory_sense;"But this kind of stimulus neighbourhood  he  engineers point of view is actually extremely interesting and give you a hell of a lot of job satisfaction because you are getting the youngsters really involved in what it's all about. And I mean we, quite obviously, we get them involved with er school trips, as a matter of fact, I've just organised a er  thing I've got here at the moment  organised a er trip  must be a good one mustn't it! Sizewell B  I mean you all heard of Sizewell, we're quite near our Sizewell. At the moment they've a very good training plant for  for Sizewell B, they show you the old erm  magnox station, but they also  the pressure water reactor station, and course also we've got the very old one down at  Bradwell  if you know about, down on the  Me  West Merseyside, we got the old power station down there."
living_sense;"Right just to finish off I just wanna look at actual vertical distributions of  elements in the ocean.  We can actually characterize  three types of distribution.  Firstly those substances which increase relatively rapidly with depth and then become constant  and this group are usually referred to as the bio-limiting  element or a  bi  having a bio-limiting distribution.  Substances which are required  nutrients, micro-nutrients, copper and zinc are required as micro-nutrients by plant cells for example, would show distributions like this   de  decreased levels in surface waters where er in the photic zone biological activity is high they'll be continually stripped out of the water column  there  in  the concentrations then increase as biological activity decreases and then becomes constant with depth."
living_sense;"If I, if I get another species next year nesting, that hasn't nested before and I know it's because of the effort of putting in reeds, trees, whatever I've put in there, we're, we're trying to get  to nest here, if they nest that'll be really, er that'll be a great thing because er they feed on the apple erm I thought somebody said er who said there was a hundred a seventy five here er a week ago and that'd be great to get them to nest, be really good.  As the ash dries hard it is dug out for sale for the construction industry, yet another habitat is created  With the  layers of  rock compacted over millions of years, this soft ash forms perfect artificial gifts for burrowing animals like rabbits  For this young fox, what better place to practise digging than a soft cliff of ash, watched nervously by potential lunch  Another predator not necessarily fond of the ash, but very keen on the animals that it attracts, is the polecat ferret  Once used by poachers many of these agile hunters are now wild and range freely over the rough ground near power stations in search of prey  Round at the top of the ash  are the nest borrows of one of Britain's rarest bird, the sand martin  the monitor of this power station has ensured special  in there so the birds can find ideal nest sites. He's even arranged for holes to be drilled into the ash to give the sand martins a head start  These days secured nest sites for these beautiful birds are increasingly rare. This artificial ash cliff is perhaps one of their last strongholds   have also found refuge around our fire stations, protected from tramping feet and sheltered from chemical sprays these rare plants thrive in the damp salty margins alongside the ash lagoons  all this within the boundaries with just a few of the country's coal fired power stations  but the need for sea walls of other coastal stations merely intake pipes to the coaling system, another world teams  this is not a plant, fan worms have flowerlike mouth parts used to filter food from the sea water  other worms use just two sticky tentacles to catch food   shrimps forage over closely packed sea  the delicate bodies of these printed vertebrates work as tiny water pumps, pulling water in one hole and pumping it through the other  here two barnacles feed in the gentle flow of water over the cooling pipes from the station  Marine life quickly packs the underwater structures and is about to become too thick and sometimes affect the performance of the machinery  barnacles belong to the same family as crabs and lobsters, but being in their adult life standing on their head which is fixed to the concrete  they use their feet to filter the water and kick food into their mouths  but if they can't move, how do they get there so quickly in the first place?"
living_sense;"History is the study of the past using documents and inscriptions as evidence, and historians have recorded and interpreted events from the earliest days of writing up to the present day. Archaeology, on the other hand, is the study of the past using objects and other excavated evidence as the main source of data. This allows the study of peoples without any form of writing, as well as of literate civilizations. Almost anything can be used to shed light on the past: the foundations of buildings can show what sort of houses people lived in; the remains of tools show how they worked; animal bones and plant remains show what they ate; and pollen, snail shells and the remains of trees and plants can indicate the type of climate and vegetation at the time."
living_sense;"There is something about taters, something deeply satisfying, even in the peeling of them. Kate puts this down to the peasant in her. Kate's family came over from Ireland a long time ago and their closest contact to the soil is the odd spot of weekend gardening, but Kate associates closely with the dispossessed peasantry of old Ireland, and anywhere else for that matter, and taters brings out the culchie in her. One day soon when she has a garden again, she will plant taters of her own, whole rows of them."
living_sense;"So generous is this bequest by birds to their young that a chick needs no additional food from which to build the flesh and bones and feathers of its infant body, and it still has enough energy left over to break its way out of the shell. Insect eggs, however, contain very little yolk. Instead, the females may help their young by placing their eggs where the minute hatchlings will find food just as soon as their heads emerge from the egg capsule. A butterfly sticks them on the leaves of the particular plant that her caterpillars eat; a blowfly on the dead flesh that her maggots will relish; and some wasps, for the sake of their young, become bodysnatchers.
factory_sense;"The most diligent student of the region's past will find it extremely difficult to fix upon any period in earlier times in which the condition of the people can be seen as more attractive than the point reached by the early 1960s. The actual industrial structure of the mid-1960s, by which time the massive Robens-led reduction in mining employment was already underway, is given in Table 3.1 and compared with the situation in 1976 and 1986. The changes over the period 196676 are interesting. The transition from the post-war boom in traditional industries to a modernized branch plant structure occurred after the Hailsham Plan of 1963 and changed the nature of the origins of the region's economic problems from overspecialization by industrial sector (i.e. over-reliance on old basic industries particularly affected by changing market demands and the new international division of labour) into overspecialization by corporate function (i.e. under-representation of control and research and development functions)."
living_sense;"Above jacket detail from The Glasshouse Garden by John Watkins, one of the Royal Horticultural Society Collection (Conran Octopus) This is the year of the publisher's themed series. In fact, such series have been building up around us gradually, until they have now become a major part of the market. Some series have considerable value  pocket guides, encyclopedias in parts, or series devoted to a specialist topic, like Cassell's foliage plant series last year."
living_sense;"(3) THE POWER OF EVERYDAY LANGUAGE The science that children encounter in school sometimes introduces domains of which children have little or no everyday experience. At other times it involves them in working with familiar phenomena. In some areas, such as vision, what  happens to the food they eat, explanations of day and night, shadows and plant growth, many children had well-established views which they had little difficulty  in expressing."
living_sense;"Here Minton would occasionally stretch out under the trees and draw.  ILLUSTRATION FOR THE SNAIL THAT CLIMBED THE EIFFEL TOWER At Lehmann's Minton met Christopher Isherwood with whom he got on well. It was probably here, too, that he first encountered Paul Odo Cross, the wealthy ex-ballet dancer who lived with the plant photographer, Angus Wilson, at Tidcombe Manor in Wiltshire, where they created a marvellous garden and influenced Cedric Morris with their growing and breeding of irises."
living_sense;"This enabled the preservation of live programmes for subsequent retransmission or sale to other countries. The telerecording screen was exactly the same as a domestic television screen except that it was flat rather than curved, eliminating any distortion from the image when captured on film. Production-wise, The Quatermass Experiment  was the same as The Time Machine , save for two small, but significant additions. As the infected astronaut begins his terrible mutation into a plant life-form the BBC's Make-up people were required to devise half-stage applications and appliances to give the creature a monsterish look."
living_sense;"Nothing worked on that one. Everything went wrong. It was a hell of a job even to complete some episodes when the money ran out. There were whole sequences of effects  rotating stone idols, descending ceilings with protruding spikes, moving plant vines whole sequences which, on a feature film, would cost a lot to set up, and take weeks, possibly months, to film. The 300dpi world will probably start to move to 400dpi fairly shortly but the vendors first need to make the move to using outline fonts rather than bitmaps. Still within the page printer world, but at higher resolution, is AM Varityper's VT-600. This is the nearest thing to typesetting yet released and it speaks PostScript so for desktop publishers looking for the quality but without typesetting it could be the answer. The system is designed for fairly low volumes so it is more suited to an art studio than in-plant work but many companies already use it instead of a phototypesetter for much of their origination." """

We're loading it as a string since we're in Colab, but files can be read similarly. To properly load in our "CSV" string, we use the following:

In [3]:
from io import StringIO
import csv

f = StringIO(corpus)
reader = csv.reader(f, quotechar='"', delimiter=';')
dataset = []
for row in reader:
    dataset.append(row)
    
# We'll also utilize NumPy, which provides us a nice way to handle arrays
import numpy as np
dataset = np.array(dataset)

# Let's just print ot the first 10
dataset[:5]

array([['factory_sense',
        'Until the mid- and late 1970s, there were simply no rules whereby groupings of parents could obtain a state-financed, multi-denominational school, as the only channels of communication in these matters were those between the Department of Education and the relevant diocesan department or other church board. For example, a new housing estate would be built, the diocese would be informed of the development at the planning stage, and the diocesan office would put in for a school, or an extension for an existing school. The arrangement had been unchanged since the early years of the state and had become entirely natural. Similarly for the protestant community: the school plant is owned by the church, or appropriate church body.'],
       ['factory_sense',
        'Bodfari Foods is a processor and supplier of liquid milk. Last year the company saw pre-tax profits of 1.9m. JUS-ROL INVESTS POTATO product manufacturer Jus-Rol has invested 1.5m in its potato ma

But before moving on, let's separate the class labels, let's call $Y$, and the documents, let's call $D$. The following code flips (transposes) the whole dataset (which is also a matrix) and gets the entire row (which prior to transposition, was a column).

In [4]:
# Transposing an array or matrix flips it on its side
# This will 

X = dataset.T[1] 
Y = dataset.T[0]

print(X[:5])
print(Y[:5])

['Until the mid- and late 1970s, there were simply no rules whereby groupings of parents could obtain a state-financed, multi-denominational school, as the only channels of communication in these matters were those between the Department of Education and the relevant diocesan department or other church board. For example, a new housing estate would be built, the diocese would be informed of the development at the planning stage, and the diocesan office would put in for a school, or an extension for an existing school. The arrangement had been unchanged since the early years of the state and had become entirely natural. Similarly for the protestant community: the school plant is owned by the church, or appropriate church body.'
 'Bodfari Foods is a processor and supplier of liquid milk. Last year the company saw pre-tax profits of 1.9m. JUS-ROL INVESTS POTATO product manufacturer Jus-Rol has invested 1.5m in its potato manufacturing plant at Amble, Northumberland. By the end of this mon

# 2. Understanding the Corpus

So now that we've loaded our data and separated the word sense tag (*class label*) from the actual context (*document*), let's look at some statisctics. **Exercise: But as a quick exercise, kindly extract the following information**:

*   Number of documents in the dataset
*   Number of living_sense labels
*   Number of factory_sense labels
*   Also calculate the distribution of each class
*   Lastly, get the total number of words (no punctuations) for each class (*this one is a little hard*)



In [5]:
# Write your code here! Feel free to search up NumPy tutorials

N = len(Y)
labels, counts = np.unique(Y, return_counts=True)
  
living_indexes, dump = np.where(dataset == 'living_sense')
factory_indexes, dump = np.where(dataset == 'factory_sense')

from nltk import word_tokenize
import string

def count_words(nArray):
  punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'
  transtab = str.maketrans(dict.fromkeys(punct, ''))
  
  word_count = 0
  for sentence in nArray:
    tokens = word_tokenize(sentence.translate(transtab))
    # the translate portion just removed the punctuation
    word_count += len(tokens)
  return word_count

living_count = count_words(X[living_indexes])
factory_count = count_words(X[factory_indexes])

print("Total document count: %s" % N)
for label, count, word_count in zip(labels, counts, [factory_count, living_count]):
  print("%s: %s (%.4f); Word count: %s (%.4f)" % (label, count, count/N, word_count, word_count/np.sum([living_count, factory_count])))

Total document count: 187
factory_sense: 101 (0.5401); Word count: 11085 (0.5776)
living_sense: 86 (0.4599); Word count: 8106 (0.4224)


**Question: What do we observe from the distribution? :)**

Let's take some time to discuss some assumptions.

This obviously doesn't tell us much, but it does give us an overview of what to expect from out data. But with that, we've successfully loaded our dataset and have a brief overview of it.


# 3. Cleaning our raw text data (Pre-processing)

**Data cleaning** is an important task in NLP as in most cases, we have horriblely structured data. Fortunately, we actually don't need to clearn our data too much since its actually quite organized. We do just need to remove some punctuations. So let's get on modifying our $X$. Since we're just removing punctuation, we can use reuse what we did awhile ago using Python's string translate.

In [6]:
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'
transtab = str.maketrans(dict.fromkeys(punct, ''))

print(X[6])

i = 0
while i < len(X):
  X[i] = X[i].translate(transtab)
  i += 1
  
print(X[6])

The head brewer at Clark's has helped out a lot, said Ryburn's owner, Harold Coburn. Although small brewers are competitors there's a lot of camaraderie. Harold started Ryburn in July 1990 after being made redundant. He used his pay-off money to buy the five barrel capacity plant.
The head brewer at Clarks has helped out a lot said Ryburns owner Harold Coburn Although small brewers are competitors theres a lot of camaraderie Harold started Ryburn in July 1990 after being made redundant He used his payoff money to buy the five barrel capacity plant


Kindly note the removal of punctuation like this does have good and bad effects on the entire corpus. Good where we get rid of unwanted features, but bad where worods like `Ryburn's` turns to `Ryburn`. Do note that we're assuming we'll lose some information here and we just need to be ready to justify it.

Cleaning of data might even refer to using lexicons to categorize words, applying POS tagging, looking at higher n-grams, etc. The basic idea is to ready the data for processing - hence why we call it **Data Pre-processing**!

# 4. Splitting data into training and testing sets

When looking to perform machine learning, we need a way to evaluate the performance of our models, so we look to split out data into training and testing sets. So here's the idea:

![alt text](https://cdn-images-1.medium.com/max/1600/1*-8_kogvwmL1H6ooN1A1tsQ.png =400x)

*source: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6*

Our **training set** becomes the knowledge we learn, but it is possible to overfit on our knowledge - or memorize the features. The problem with memorization is that its hard to generalize unless its an exact match. So to test if our learned knowledge is generalizable, we get data that we have not seen / have not learned from and test - hence, the **test set**! There are also other concepts needed to be learned for the ML side, but we'll only tackle the bare minimum.

So on to some code! Let's first split our data into training and test. We'll set our testing size to 30% of our total dataset, leaving 70% for training. These numbers can be changed around, but for this session, these numbers should be good.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, # Features
    Y, # Labels
    test_size = 0.3, # The defined test size; Training size is just 1 minus the test size
    random_state = 12 # So we can shuffle the elements, but with some consistency
)

print("=====\nTraining Data")
print("Document count: %s" % len(Y_train))
labels, counts = np.unique(Y_train, return_counts=True)
for label, count in zip(labels, counts):
  print("%s: %s (%.4f)" % (label, count, count/len(Y_train)))
        
print("=====\nTesting Data")
print("Document count: %s" % len(Y_test))
labels, counts = np.unique(Y_test, return_counts=True)
for label, count in zip(labels, counts):
  print("%s: %s (%.4f)" % (label, count, count/len(Y_test)))

=====
Training Data
Document count: 130
factory_sense: 66 (0.5077)
living_sense: 64 (0.4923)
=====
Testing Data
Document count: 57
factory_sense: 35 (0.6140)
living_sense: 22 (0.3860)


By default `train_test_split` performs a stratified split based on the classes.
Stratify just means to arrange/sort into groups, so with our data, it balances
our the 2 classes (living versus factory). 

See more about the function here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

**Question: What would happen if we just randomly got indexes from the whole dataset? What implications could happen if we performed stratification or not?**

# 5. Extracting features from text

We discussed two ways to extract information in class: collocation and bag-of-words. Collocation looks at the words $\pm N$ from an anchor word at index $i$. We can typically look at the words and their POS tag. For bag-of-words, we don't care about the order and look at what words are found in the document. Both representations can be shown in the form of a vector, so in this session, we'll just focus on extracting a bag-of-words model. Let's look at sklearn's `CountVectorizer`

**Remember! Don't touch the testing data yet!**

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer    

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Some parameters to note:


*   Analyzer - We can analyze at the **word** or **char**(acter) level
*   Binary - If binary is set to true, it will count if the word appeared or not (0 or 1). If set to false, it will count how many times the word appeared.
*   Max_df & min_df - Maximum and minimum document frequency. If a word is too common or too unique with respect to appearing across documents, it can cause noise. This can be solved by setting the min and max document frequencies.
*   Max_features - Considers the top max_features ordered by term frequency
*   Ngram_range - (1, 1) means we're only looking at 1-grams. (1, 2) means include 1 and 2-grams. Think of it as min and max.
*   Tokenizer - you can set this as your own function for tokenizing!
*   Vocabulary - You can set your own vocabulary here. If not, then it'll consider all words.

For more info, check out their site: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

So let's trying running it on our training data.



In [9]:
import pandas as pd

x = vectorizer.fit_transform(X_train)
count_vect_df = pd.DataFrame(x.todense(), columns=vectorizer.get_feature_names())

count_vect_df.head()

,000,10,100,1000,104in,107,1086,109,10pm,11,...,you,youll,young,youngsters,your,youre,zandvoort,zinc,zip,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Before anything else, this is **Pandas**, easy-to-use data structures and data analysis tools for the Python. Think of it as an abstraction over NumPy and instead of treating your datasets as arrays, we can think of them as dataframes.

So in the code above, we used our count vectorizer and passed our training data on it. Fit transform just means, fit the vectorizer based on the vocab of the training data, so we can use a simple transform on the testing data later.

From the output data frame, we can tell that there are 4155 features (since there are 4155 columns). Each column is a feature and each row is a document. There are only 5 rows in the output because we printed only the head of the DataFrame.

**Question: Looking at our data, what is an issue here?**

(some) Answer:

*   **Number of features** - We have a load of features! 4k plus with only 130 training instances! This may be generalizable because we're account for many possible features, but we're seeing a lot of information. What happens if we expand out training samples, will out feature size grow? Yup!
*   **Sparcity** - Because we have a lot of features, we're to expect a lot of zeros as well. And if there is only one entry for a specific feature, it might not be super useful.

Let's look at how different configurations can affect the dataset.



In [10]:
print("Default:", CountVectorizer().fit_transform(X_train).shape)
print("Min df set to 1%:", CountVectorizer(min_df=.01).fit_transform(X_train).shape)
print("Min df set to 5%:", CountVectorizer(min_df=.05).fit_transform(X_train).shape)
print("Min df (1%), Max df (90%):", CountVectorizer(max_df=.9, min_df=.01).fit_transform(X_train).shape)
print("Using max features (50):", CountVectorizer(max_features=50).fit_transform(X_train).shape)
print("Using 1,2,3-grams:", CountVectorizer(ngram_range=(1,3)).fit_transform(X_train).shape)
print("Using 1,2,3-grams, Min df (1%), Max df (90%):", CountVectorizer(max_df=.9, min_df=.01, ngram_range=(1,3)).fit_transform(X_train).shape)
print("Using a defined vocabulary:", CountVectorizer(vocabulary=['meme', 'of', 'the', 'week']).fit_transform(X_train).shape)
print("Lowercase off:", CountVectorizer(lowercase=False).fit_transform(X_train).shape)

Default: (130, 4155)
Min df set to 1%: (130, 1237)
Min df set to 5%: (130, 181)
Min df (1%), Max df (90%): (130, 1234)
Using max features (50): (130, 50)
Using 1,2,3-grams: (130, 27970)
Using 1,2,3-grams, Min df (1%), Max df (90%): (130, 2255)
Using a defined vocabulary: (130, 4)
Lowercase off: (130, 4455)


See how the configurations affect the number of features we can learn from? There can be many ways to create representations that our algorithms can learn from. We just need to be smart and justify why our configurations should be considered.

Just note that extracting features is key! Instead of just counts, you can consider:

*   Term Frequency (TF)
*   Term Frequency Inverse Document Frequency (TFIDF)
*   Binary counts
*   POS counts
*   Lexicon counts

Explore the different levels of information we can extract from text. Some might not be as useful, while others will be fit for the problem.



# 6. Learning based on the features / Training a model / Testing a model

Now that we're able to extract information, let's start with our models. We won't go into detail with the models, but we'll consider the following:

*   Naive Bayes (NB)
*   k-Nearest Neighbors (kNN)
*   Support Vector Machines (SVM)

All ML algorithms have their own strentghs (training time, algorithmic complexicty, etc), so its yet another factor to consider when dealing with classification. So let's formalize our feature set as follows


In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, # Our training set
    Y, # Our test set
    test_size = 0.3, # The defined test size; Training size is just 1 minus the test size
    random_state = 12 # So we can shuffle the elements, but with some consistency
)

# We'll just use simple counts
vectorizer = CountVectorizer(
    max_df=.9, 
    min_df=.01,
    ngram_range=(1,1),
    binary=True
)

print("=====FIT TRANSFORM=====")
X_train = vectorizer.fit_transform(X_train)
print(X_train)
X_test = vectorizer.transform(X_test)
print("=====X_TEST=====")


=====FIT TRANSFORM=====
  (0, 164)	1
  (0, 687)	1
  (0, 477)	1
  (0, 143)	1
  (0, 96)	1
  (0, 795)	1
  (0, 452)	1
  (0, 1059)	1
  (0, 468)	1
  (0, 34)	1
  (0, 796)	1
  (0, 188)	1
  (0, 722)	1
  (0, 551)	1
  (0, 284)	1
  (0, 855)	1
  (0, 616)	1
  (0, 756)	1
  (0, 1001)	1
  (0, 1098)	1
  (0, 951)	1
  (0, 308)	1
  (0, 82)	1
  (0, 1044)	1
  (0, 970)	1
  :	:
  (129, 122)	1
  (129, 982)	1
  (129, 918)	1
  (129, 928)	1
  (129, 1086)	1
  (129, 1141)	1
  (129, 748)	1
  (129, 269)	1
  (129, 935)	1
  (129, 394)	1
  (129, 1092)	1
  (129, 837)	1
  (129, 1192)	1
  (129, 581)	1
  (129, 543)	1
  (129, 1188)	1
  (129, 452)	1
  (129, 722)	1
  (129, 551)	1
  (129, 782)	1
  (129, 83)	1
  (129, 119)	1
  (129, 437)	1
  (129, 1112)	1
  (129, 1084)	1
=====X_TEST=====


Let's look at the following lines of code:

*   `vectorizer.fit_transform(X_train)` - This function fits and transforms the data. Fit means to understand the data passed in. We passed `X_train`, so its like a "oh this is what the training data looks like, let's get its vocab!" Transform means, whatever the data was that was passed in, transform it based on the fit. This returns a fitted count matrix. By context, that also means you can peform a `vectorizer.fit(X_train)` which just fits the data.
*   `vectorizer.transform(X_test)` - This function just transforms based on the fit and since our vectorizer was already fitted with the `X_train`, we now just fit the `X_test` to the `X_train` vocab.

Now let's get to actual classification!



In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# The three classifiers that we'll use
clfs = [
    KNeighborsClassifier(n_neighbors=5), 
    MultinomialNB(),
    svm.SVC(kernel='linear')
]

for clf in clfs:
  clf.fit(X_train, Y_train)
  y_pred = clf.predict(X_test)
  acc = accuracy_score(Y_test, y_pred)
  f1 = f1_score(Y_test, y_pred, pos_label='factory_sense')  
  
  print("%s\nAccuracy: %s\nF1 Score: %s\n=====" % (clf, acc, f1))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
Accuracy: 0.49122807017543857
F1 Score: 0.32558139534883723
=====
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy: 0.8070175438596491
F1 Score: 0.8450704225352113
=====
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Accuracy: 0.7017543859649122
F1 Score: 0.7462686567164178
=====


Look! The Naive Bayes implementation performed best! This means that we're able to correctly predict up to 80.7% accuracy whether a given sentence that uses plant as either the living sense or the factory sense. Not bad, but not good.

**HOWEVER**, this performance is because we didn't perform cross-validation and actually tune our models. These are all things that will be covered outside of NLP, but are very important to gain a grasp of classification, in general. Our models have many parameters, so we need to tune them to our training data and hope they can generalize when we test our the testing set. 

But at least for now, we've performed the bare minimum for text classification. We've loaded the corpus in, extracted information, learned from the model, and tested it out on unseen data.

The assignment will be uploaded via Canvas, but you'll make sure of this Python notebook in exploring the effects of pre-processing, feature extraction, and classification.

# 7. Memes of the week

Cause you all are awesome, here are the memes of the week

![alt text](https://scontent.fmnl10-1.fna.fbcdn.net/v/t1.15752-9/52458975_2541773996049221_2083917586558353408_n.png?_nc_cat=100&_nc_ht=scontent.fmnl10-1.fna&oh=b484b4da148a84fe6a039e0d7cc97e27&oe=5CF6C184)

Meme submitted by Blaise Cruz (yeaboi)

![alt text](https://scontent.fmnl10-1.fna.fbcdn.net/v/t1.0-9/49209747_520866785074567_581292375763058688_n.png?_nc_cat=101&_nc_ht=scontent.fmnl10-1.fna&oh=005e4c0848f79f80bab633c142da65f4&oe=5CDE8ADE)